In [ ]:
%env OPENAI_API_KEY = api_key

In [ ]:
# pip install --upgrade openai

In [ ]:
import glob
import json
import re
import random
import time

from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
cd '/home/azmain/alljavajsons'

In [ ]:
inputFiles = []
correctOutputFiles = []

    
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

for file in sorted(glob.glob("*.java.json"), key=numericalSort):
    inputFiles.append(file)

for file in sorted(glob.glob("*.benchmark_log.json"), key=numericalSort):
    correctOutputFiles.append(file)

print(inputFiles)
print('\n\n\n')
print(correctOutputFiles)

In [ ]:
def get_codes(inputFiles):
    codes = []
    for code in inputFiles:
        codes.append(str(json.load(open(code))['originalContent']))

    return codes

In [ ]:
def get_correct_outputs(correctOutputFiles):
    correct_outputs = []
    for output in correctOutputFiles:
        correct_output_list = json.load(open(output))['total_imports']
        correct_outputs.append(correct_output_list)
    
    for import_lines in correct_outputs:
        if "gen.R" in import_lines:
            import_lines.remove("gen.R")

    return correct_outputs

In [ ]:
def get_dataset(codes, correct_outputs):
    dataset = {
        "codes": codes,
        "correct_outputs": correct_outputs
    }
    return dataset

In [ ]:
def get_test_examples_and_y_true(dataset):
    sample_list = []

    for i in range(0, len(dataset["codes"])):
        sample_list.append(dict(codes=dataset["codes"][i], correct_outputs=dataset["correct_outputs"][i]))
    
    # print(sample_list)
    
    test_examples = [(example["codes"], example["correct_outputs"]) for example in sample_list]
    y_true = [correct_outputs for _, correct_outputs in test_examples]
    
    return test_examples, y_true

### All Android Classes

In [ ]:
print("Total Android Codes: {}\n".format(len(inputFiles[:50])))
print(inputFiles[:50])

android_codes = get_codes(inputFiles[:50])
# print(android_codes)

android_correct_outputs = get_correct_outputs(correctOutputFiles[:50])
# print(android_correct_outputs)

android_dataset = get_dataset(android_codes, android_correct_outputs)
# print(android_dataset)

android_test_examples, android_y_true = get_test_examples_and_y_true(android_dataset)
# print(android_test_examples)

### All JDK Classes

In [ ]:
print("Total JDK Codes: {}\n".format(len(inputFiles[50:73])))
print(inputFiles[50:73])

jdk_codes = get_codes(inputFiles[50:73])
# print(jdk_codes)

jdk_correct_outputs = get_correct_outputs(correctOutputFiles[50:73])
# print(jdk_correct_outputs)

jdk_dataset = get_dataset(jdk_codes, jdk_correct_outputs)
# print(jdk_dataset)

jdk_test_examples, jdk_y_true = get_test_examples_and_y_true(jdk_dataset)
# print(jdk_test_examples)

### All Hibernate Classes

In [ ]:
print("Total Hibernate Codes: {}\n".format(len(inputFiles[73:74] + inputFiles[174:224])))
print(inputFiles[73:74] + inputFiles[174:224])

hibernate_codes = get_codes(inputFiles[73:74] + inputFiles[174:224])
# print(hibernate_codes)

hibernate_correct_outputs = get_correct_outputs(correctOutputFiles[73:74] + correctOutputFiles[174:224])
# print(hibernate_correct_outputs)

hibernate_dataset = get_dataset(hibernate_codes, hibernate_correct_outputs)
# print(hibernate_dataset)

hibernate_test_examples, hibernate_y_true = get_test_examples_and_y_true(hibernate_dataset)
# print(hibernate_test_examples)

### All JodaTime Classes

In [ ]:
print("Total JodaTime Codes: {}\n".format(len(inputFiles[74:124])))
print(inputFiles[74:124])

jodatime_codes = get_codes(inputFiles[74:124])
# print(jodatime_codes)

jodatime_correct_outputs = get_correct_outputs(correctOutputFiles[74:124])
# print(jodatime_correct_outputs)

jodatime_dataset = get_dataset(jodatime_codes, jodatime_correct_outputs)
# print(jodatime_dataset)

jodatime_test_examples, jodatime_y_true = get_test_examples_and_y_true(jodatime_dataset)
# print(jodatime_test_examples)

### All GWT Classes

In [ ]:
print("Total GWT Codes: {}\n".format(len(inputFiles[124:174])))
print(inputFiles[124:174])

gwt_codes = get_codes(inputFiles[124:174])
# print(gwt_codes)

gwt_correct_outputs = get_correct_outputs(correctOutputFiles[124:174])
# print(gwt_correct_outputs)

gwt_dataset = get_dataset(gwt_codes, gwt_correct_outputs)
# print(gwt_dataset)

gwt_test_examples, gwt_y_true = get_test_examples_and_y_true(gwt_dataset)
# print(gwt_test_examples)

### All XStream Classes

In [ ]:
print("Total XStream Codes: {}\n".format(len(inputFiles[224:268])))
print(inputFiles[224:268])

xstream_codes = get_codes(inputFiles[224:268])
# print(xstream_codes)

xstream_correct_outputs = get_correct_outputs(correctOutputFiles[224:268])
# print(xstream_correct_outputs)

xstream_dataset = get_dataset(xstream_codes, xstream_correct_outputs)
# print(xstream_dataset)

xstream_test_examples, xstream_y_true = get_test_examples_and_y_true(xstream_dataset)
# print(xstream_test_examples)

# Zero-shot and Self-Consistency Implementation

In [ ]:
MODEL = 'gpt-3.5-turbo'

In [ ]:
def pred_process(y_pred, y_true):
    y_pred_processed = []
    y_true_processed = []
    
    for pred, correct_imports in zip(y_pred, y_true):
        max_length = max(len(pred), len(correct_imports))
        correct_preds = list(set(pred).intersection(correct_imports))
        for i in range(0, max_length):
            if i<len(correct_preds):
                y_pred_processed.append(1)
                y_true_processed.append(1)
            else:
                if i<len(correct_imports):
                    y_pred_processed.append(0)
                    y_true_processed.append(1)
                else:
                    y_pred_processed.append(1)
                    y_true_processed.append(0)
            
    print(y_pred_processed)
    print(y_true_processed)
    print()
    return y_pred_processed, y_true_processed

In [ ]:
def eval_performance(y_pred, y_true):
    print(json.dumps({
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred)
    }, indent=2))

In [ ]:
# # Test

# print(xstream_test_examples[7][1]) # correct imports
# print(xstream_test_examples[7][0]) # code

In [ ]:
# # Test

# MODEL = 'gpt-3.5-turbo'

# code_snippet = xstream_test_examples[7][0]
# # prompt=f"Reply with to-the-point answer, with no elaboration. Extract all valid possible fully qualified type long-name from the code below, must exclude name of the class itself:\n\"\"\"{test_code}\"\"\""

# response = openai.ChatCompletion.create(
#     model=MODEL,
#     messages=[
#         {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
# #         {"role": "user", "content": f"Import correct imports:\n\"\"\"\n{test_code}\n\"\"\""},
# #         {"role": "user", "content": f"Your task is to provide a list of the correct imports for a given code snippet, ensuring that no wildcard imports are used. If any necessary imports are missing from the code, please include them in your response. Please note that your response should be specific and accurate:\n\"\"\"\n{code_snippet}\n\"\"\""},
# #         {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that your response should be specific and accurate:\n\"\"\"\n{code_snippet}\n\"\"\""},
# #         {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention, and your response should be extremely specific and highly accurate. You are prohibited from importing wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""},
# #         {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention, and your response should be specific and accurate. Do not generate wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""},
#         {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""}
#     ],
#     temperature=0,
# )

# print(response["choices"][0]["message"]["content"])

# Zero-shot

In [ ]:
def get_prediction(code_snippet):
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
            {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""}
        ],
        temperature=0.5,
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
def get_predictions(dataset):
    y_pred = []
    for code_snippet, correct_imports in tqdm(dataset):
        predicted_output = re.findall(r'\b[a-zA-Z0-9]+(?:\.[a-zA-Z0-9]+)+\b', get_prediction(code_snippet))
        y_pred.append(predicted_output)
        time.sleep(5)
    return y_pred

In [ ]:
# Prediction for Android Classes

print("\nPrediction for Android Classes:\n")
y_pred = get_predictions(android_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", android_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction for JDK Classes

print("\nPrediction for JDK Classes:\n")
y_pred = get_predictions(jdk_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jdk_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction for Hibernate Classes

print("\nPrediction for Hibernate Classes:\n")
y_pred = get_predictions(hibernate_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", hibernate_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction for Joda-Time Classes

print("\nPrediction for Joda-Time Classes:\n")
y_pred = get_predictions(jodatime_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jodatime_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction for GWT Classes

print("\nPrediction for GWT Classes:\n")
y_pred = get_predictions(gwt_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", gwt_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction for XStream Classes

print("\nPrediction for XStream Classes\n")
y_pred = get_predictions(xstream_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", xstream_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)

## Self-Consistency with Modified Parameters (Default 10 Sample)

In [ ]:
def get_modified_temp_prediction(code_snippet):
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
            {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""}
        ],
        temperature=1,
        top_p=1,
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
from collections import Counter

def get_consistent_item(k_items_list, code_snippet):
    
    # Count the frequency of each set of sublists
    counts = Counter(frozenset(sublist) for sublist in k_items_list)

    # Find the maximum frequency count
    max_count = max(counts.values())

    # Find all sets of sublists with the maximum frequency count
    most_common_sublists_set = {frozenset(sublist) for sublist in counts if counts[sublist] == max_count}

    # Convert the sets of sublists back to lists
    most_common_sublists = [list(sublist) for sublist in most_common_sublists_set]

    # Find the number of sublists that have the highest frequency count
    max_consistency_count = len(most_common_sublists)
    
    max_check_time = 10
    i = 0
    while max_consistency_count!=1:
        if i > max_check_time:
            return []
        time.sleep(2)
        new_predicted_output = re.findall(r'\b[a-zA-Z0-9]+(?:\.[a-zA-Z0-9]+)+\b', get_modified_temp_prediction(code_snippet))
        k_items_list.append(new_predicted_output)
        counts = Counter(frozenset(sublist) for sublist in k_items_list)
        max_count = max(counts.values())
        most_common_sublists_set = {frozenset(sublist) for sublist in counts if counts[sublist] == max_count}
        most_common_sublists = [list(sublist) for sublist in most_common_sublists_set]
        max_consistency_count = len(most_common_sublists)
        i+=1

    consistent_item = most_common_sublists[0]
    
    return consistent_item

In [ ]:
def get_all_predictions(dataset, k):
    y_pred_list = []
    for code_snippet, correct_imports in tqdm(dataset):
        k_output_sample = []
        for i in range(0, k):
            time.sleep(5)
            predicted_output = re.findall(r'\b[a-zA-Z0-9]+(?:\.[a-zA-Z0-9]+)+\b', get_modified_temp_prediction(code_snippet))
            k_output_sample.append(predicted_output)
        y_pred_list.append(get_consistent_item(k_output_sample, code_snippet))
    return y_pred_list

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for Android Classes

print("\nPrediction with Self-Consistency and Modified Parameters for Android Classes\n")
k = 10
y_pred = get_all_predictions(android_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", android_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for Android Classes from 20 Samples 

print("\nPrediction with Self-Consistency and Modified Parameters for Android Classes from 20 Samples\n")
k = 20
y_pred = get_all_predictions(android_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", android_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for Android Classes from 30 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for Android Classes from 30 Samples\n")
k = 30
y_pred = get_all_predictions(android_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", android_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for JDK Classes

print("\nPrediction with Self-Consistency and Modified Parameters for JDK Classes\n")
k = 10
y_pred = get_all_predictions(jdk_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jdk_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for JDK Classes from 20 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for JDK Classes from 20 Samples\n")
k = 20
y_pred = get_all_predictions(jdk_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jdk_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for JDK Classes from 30 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for JDK Classes from 30 Samples\n")
k = 30
y_pred = get_all_predictions(jdk_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jdk_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for Hibernate Classes

print("\nPrediction with Self-Consistency and Modified Parameters for Hibernate Classes\n")
k = 10
y_pred = get_all_predictions(hibernate_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", hibernate_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for Hibernate Classes from 20 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for Hibernate Classes from 20 Samples\n")
k = 20
y_pred = get_all_predictions(hibernate_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", hibernate_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for Hibernate Classes from 30 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for Hibernate Classes from 30 Samples\n")
k = 30
y_pred = get_all_predictions(hibernate_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", hibernate_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for Joda-Time Classes

print("\nPrediction with Self-Consistency and Modified Parameters for Joda-Time Classes\n")
k = 10
y_pred = get_all_predictions(jodatime_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jodatime_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for Joda-Time Classes from 20 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for Joda-Time Classes from 20 Samples\n")
k = 20
y_pred = get_all_predictions(jodatime_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jodatime_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for Joda-Time Classes from 30 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for Joda-Time Classes from 30 Samples\n")
k = 30
y_pred = get_all_predictions(jodatime_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jodatime_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for GWT Classes

print("\nPrediction with Self-Consistency and Modified Parameters for GWT Classes\n")
k = 10
y_pred = get_all_predictions(gwt_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", gwt_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for GWT Classes from 20 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for GWT Classes from 20 Samples\n")
k = 20
y_pred = get_all_predictions(gwt_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", gwt_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for GWT Classes from 30 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for GWT Classes from 30 Samples\n")
k = 30
y_pred = get_all_predictions(gwt_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", gwt_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for XStream Classes

print("\nPrediction with Self-Consistency and Modified Parameters for XStream Classes\n")
k = 10
y_pred = get_all_predictions(xstream_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", xstream_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for XStream Classes from 20 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for XStream Classes from 20 Samples\n")
k = 20
y_pred = get_all_predictions(xstream_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", xstream_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Self-Consistency and Modified Parameters for XStream Classes from 30 Samples

print("\nPrediction with Self-Consistency and Modified Parameters for XStream Classes from 30 Samples\n")
k = 30
y_pred = get_all_predictions(xstream_test_examples, k)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", xstream_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)

# Appending Predicted Imports to the Code Body

In [ ]:
def group_imports(predicted_imports):
    added_imports = []
    for import_group in predicted_imports:
        new_group = []
        for import_statement in import_group:
            new_group.append('import ' + import_statement + ';')
        added_imports.append(new_group)
    return added_imports

In [ ]:
def append_imports(save_directory, code_name, list_of_java_codes, list_of_imports):
    code_import_dict = {}
    # Check if lengths match
    if len(list_of_java_codes) != len(list_of_imports):
        print("Mismatch between number of Java code strings and import lists!")
    else:
        # Zip the lists together into a dictionary
        code_import_dict = {i: (imports, code) for i, (imports, code) in enumerate(zip(list_of_imports, list_of_java_codes))}

    # Prepend the imports to the Java codes, save each to a .java file
    for index, (imports, code) in code_import_dict.items():
        # Split the code into lines
        lines = code.split('\n')
        
        # Find the line with the package declaration
        package_line_index = next((i for i, line in enumerate(lines) if line.strip().startswith('package ')), None)

        # If a package declaration is found, insert the imports after it
        if package_line_index is not None:
            lines = lines[:package_line_index+1] + imports + lines[package_line_index+1:]
        else:
            # If not, prepend the imports to the code
            lines = imports + lines

        full_code = '\n'.join(lines)
        file_name = ""
        if code_name == "android":
            if index<9:
                file_name = f"Android0{index+1}.java"
            else:
                file_name = f"Android{index+1}.java"
        elif code_name == "jdk":
            file_name = f"Class_{index+1}.java" 
        elif code_name == "hibernate":
            if index == 0:
                file_name = f"HibernateUtil.java"
            else:
                file_name = f"hibernate_class_{index}.java"
        elif code_name == "jodatime":
            if index<9:
                file_name = f"JodaTime0{index+1}.java"
            else:
                file_name = f"JodaTime{index+1}.java"
        elif code_name == "gwt":
            file_name = f"gwt_class_{index+1}.java"
        elif code_name == "xstream":
            file_name = f"xstream_class_{index+1}.java"
        full_path = os.path.join(save_directory, file_name)

        # Save the full code to a .java file
        with open(full_path, "w") as f:
            f.write(full_code)

### Without Self-consistency Temp 0.5

In [ ]:
# Android Zero-shot without Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5"
code_name = "android"
codes = android_codes
predicted_imports = [['android.app.Activity', 'android.os.Bundle', 'android.widget.TextView'], ['android.app.Activity', 'android.os.Bundle'], ['android.graphics.drawable.Drawable', 'com.google.android.maps.ItemizedOverlay', 'com.google.android.maps.OverlayItem', 'com.google.android.maps.GeoPoint'], ['android.app.Activity', 'android.os.Bundle', 'android.widget.TextView'], ['android.graphics.drawable.Drawable', 'android.widget.TabHost', 'android.widget.TabSpec'], ['android.widget.LinearLayout', 'android.view.ViewGroup', 'android.view.Gravity'], ['android.app.Activity', 'android.os.Bundle', 'android.view.View', 'android.widget.ArrayAdapter'], ['java.io.IOException', 'java.net.Inet4Address', 'java.net.InetAddress', 'java.net.Socket', 'java.net.UnknownHostException', 'android.util.Log'], ['android.os.Bundle', 'android.widget.TextView', 'java.util.Timer', 'java.util.TimerTask'], ['android.os.SystemClock', 'android.util.Log', 'android.widget.Chronometer'], ['android.content.Context', 'android.graphics.Canvas', 'android.location.Location', 'android.maps.MapView', 'android.overlay.MyLocationOverlay', 'com.google.android.maps.GeoPoint'], ['android.app.Activity', 'android.graphics.Color', 'android.os.Bundle', 'android.view.ViewGroup', 'android.widget.LinearLayout', 'android.widget.TextView'], ['android.content.ContentResolver', 'android.graphics.Bitmap', 'android.graphics.BitmapFactory', 'android.net.Uri', 'java.io.ByteArrayInputStream', 'java.io.ByteArrayOutputStream', 'java.io.FileNotFoundException', 'java.io.IOException'], ['java.io.File', 'java.io.FileNotFoundException', 'java.io.FileOutputStream', 'java.io.IOException', 'java.io.InputStream', 'java.net.HttpURLConnection', 'java.net.MalformedURLException', 'java.net.ProtocolException', 'java.net.URL'], ['android.content.Intent', 'android.net.Uri'], ['android.content.Context', 'android.graphics.Canvas', 'android.util.AttributeSet', 'android.view.MotionEvent', 'android.view.View', 'android.widget.SeekBar', 'android.util.Log'], ['org.apache.http.client.HttpClient', 'org.apache.http.client.methods.HttpGet', 'org.apache.http.impl.client.DefaultHttpClient', 'org.apache.http.HttpResponse', 'org.apache.http.HttpEntity', 'java.io.IOException', 'java.io.InputStreamReader', 'java.net.URI', 'java.net.URISyntaxException', 'android.util.Log'], ['android.app.Activity', 'android.content.DialogInterface', 'android.content.Intent', 'android.os.Bundle', 'android.view.View.OnClickListener'], ['android.content.Context', 'android.view.View', 'android.view.ViewGroup', 'android.widget.GridView', 'android.widget.ImageView'], ['android.content.Context', 'android.view.View', 'android.view.animation.ScaleAnimation', 'android.widget.TextView'], ['android.content.Context', 'android.widget.ImageButton', 'android.widget.LinearLayout'], ['android.app.ListActivity', 'android.content.Intent', 'android.database.Cursor', 'android.os.Bundle', 'android.provider.Contacts.People', 'android.view.View', 'android.widget.ListView', 'android.widget.SimpleCursorAdapter'], ['android.app.Activity', 'android.graphics.Bitmap', 'android.graphics.BitmapFactory', 'android.os.Bundle', 'android.widget.ArrayAdapter', 'android.widget.ImageView', 'android.widget.ListView', 'java.io.IOException', 'java.net.HttpURLConnection', 'java.net.MalformedURLException', 'java.net.URL', 'java.util.ArrayList', 'java.util.Random'], ['android.app.Activity', 'android.content.Context', 'android.location.LocationManager', 'android.os.Bundle', 'android.util.Log', 'java.util.Iterator', 'java.util.List'], ['android.app.ListActivity', 'android.content.Context', 'android.database.sqlite.SQLiteDatabase', 'android.os.Bundle', 'java.util.ArrayList'], ['android.content.Context', 'android.view.KeyEvent', 'android.view.MotionEvent', 'android.view.View', 'android.view.View.OnTouchListener', 'android.widget.ScrollView'], ['android.content.Context', 'android.view.View', 'android.view.ViewGroup', 'android.widget.BaseAdapter', 'android.widget.GridView', 'android.widget.TextView'], ['android.content.Context', 'android.app.Application'], ['android.app.Service', 'android.content.Context', 'android.content.Intent', 'android.os.IBinder', 'android.os.PowerManager'], ['org.apache.http.HttpEntity', 'org.apache.http.HttpResponse', 'org.apache.http.client.HttpClient', 'org.apache.http.client.methods.HttpGet', 'org.apache.http.impl.client.DefaultHttpClient', 'org.apache.http.params.HttpParams', 'org.apache.http.params.BasicHttpParams', 'org.apache.http.HttpHost', 'org.apache.http.util.EntityUtils', 'java.io.IOException'], ['android.os.PowerManager', 'android.content.Context'], ['java.io.IOException', 'android.os.Environment', 'java.io.File', 'android.media.MediaRecorder'], ['android.content.Context', 'android.hardware.Camera', 'android.hardware.Camera.Parameters', 'android.graphics.PixelFormat', 'android.view.SurfaceHolder', 'java.io.IOException'], ['android.app.TabActivity', 'android.os.Bundle', 'android.util.Log', 'android.view.View', 'android.view.View.OnClickListener', 'android.widget.TabHost', 'android.widget.TabHost.OnTabChangeListener', 'android.widget.TabHost.TabContentFactory', 'android.widget.ListView'], ['android.media.MediaRecorder', 'java.io.File', 'java.io.IOException'], ['android.content.ContentValues', 'android.database.sqlite.SQLiteDatabase'], ['android.app.Activity', 'android.app.ProgressDialog', 'android.os.Handler', 'android.os.Message'], [], ['android.app.Activity', 'android.os.Bundle', 'android.widget.LinearLayout', 'com.google.android.maps.MapActivity', 'com.google.android.maps.MapController', 'com.google.android.maps.MapView', 'com.google.android.maps.MyLocationOverlay', 'com.google.android.maps.Overlay'], ['android.app.Activity', 'android.os.Bundle', 'android.widget.SlidingDrawer', 'android.widget.ImageView', 'android.view.View', 'android.view.View.OnClickListener', 'android.util.Log'], ['android.content.Context', 'android.graphics.Color', 'android.view.View', 'android.widget.TextView'], ['android.app.Activity', 'android.content.Intent', 'android.content.pm.ActivityInfo', 'android.graphics.PixelFormat', 'android.media.MediaPlayer', 'android.net.Uri', 'android.os.Bundle', 'android.util.Log', 'android.view.SurfaceHolder', 'android.view.View', 'android.webkit.URLUtil', 'android.widget.Button', 'android.widget.MediaController', 'android.widget.VideoView', 'java.io.File', 'java.io.FileOutputStream', 'java.io.IOException', 'java.io.InputStream', 'java.net.URL', 'java.net.URLConnection'], ['org.apache.http.HttpHost', 'org.apache.http.HttpResponse', 'org.apache.http.client.HttpClient', 'org.apache.http.client.methods.HttpGet', 'org.apache.http.entity.HttpEntity', 'org.apache.http.impl.client.DefaultHttpClient', 'org.apache.http.util.EntityUtils', 'java.io.IOException'], ['android.app.Service', 'android.content.Intent', 'android.os.IBinder'], ['android.app.Activity', 'android.content.BroadcastReceiver', 'android.content.ComponentName', 'android.content.Context', 'android.content.Intent', 'android.content.IntentFilter', 'android.os.Bundle', 'android.util.Log', 'android.content.IntentFilter.MalformedMimeTypeException'], ['android.app.Activity', 'android.content.Intent', 'android.os.Bundle', 'android.view.View', 'android.widget.Button', 'android.widget.EditText'], ['android.app.Activity', 'android.os.Bundle', 'android.widget.TextView', 'org.apache.http.client.ClientProtocolException', 'org.apache.http.client.methods.HttpGet', 'org.apache.http.impl.client.DefaultHttpClient', 'java.io.IOException'], ['android.hardware.Camera', 'android.hardware.Camera.PictureCallback', 'android.content.Context', 'android.app.Dialog', 'android.graphics.BitmapFactory', 'android.graphics.Bitmap', 'android.widget.TextView', 'android.widget.ImageView', 'android.app.AlertDialog'], ['android.content.Context', 'android.util.AttributeSet', 'android.view.Gravity', 'android.view.LayoutInflater', 'android.widget.LinearLayout', 'androidExamples.R'], ['android.app.Activity', 'android.os.Bundle', 'android.widget.EditText', 'android.widget.TextView', 'android.widget.Button', 'android.view.View', 'android.util.Log', 'java.text.NumberFormat']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# JDK Zero-shot without Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5"
code_name = "jdk"
codes = jdk_codes
predicted_imports = [['java.awt.Applet', 'java.awt.Color', 'java.awt.Container', 'java.awt.Dimension', 'java.awt.Frame', 'java.awt.Graphics', 'java.awt.Graphics2D', 'java.awt.event.WindowAdapter', 'java.awt.event.WindowEvent', 'java.awt.image.BufferedImage', 'java.io.FileInputStream', 'java.io.FileOutputStream', 'java.io.IOException', 'java.io.ObjectInputStream', 'java.io.ObjectOutputStream'], ['javax.swing', 'javax.swing.border.TitledBorder', 'java.awt', 'java.awt.event.ActionEvent', 'java.awt.event.ActionListener', 'java.awt.image.BufferedImage'], ['java.text.DecimalFormat', 'java.text.DecimalFormatSymbols', 'java.text.ParseException', 'java.math.RoundingMode', 'java.util.Locale'], ['java.util.regex.Pattern', 'java.util.regex.Matcher', 'java.util.ArrayList', 'java.util.List'], ['java.util.Map', 'java.util.HashMap', 'java.util.List', 'java.util.LinkedList', 'java.util.Collections', 'java.util.Comparator', 'java.util.LinkedHashMap', 'java.util.TreeMap'], ['java.awt.BorderLayout', 'java.awt.Color', 'java.awt.Component', 'java.awt.EventQueue', 'java.awt.Graphics', 'java.awt.Graphics2D', 'java.awt.RenderingHints', 'java.awt.event.ActionEvent', 'java.awt.event.ActionListener', 'java.util.Observable', 'java.util.Observer', 'java.util.Random', 'javax.swing.Icon', 'javax.swing.JButton', 'javax.swing.JFrame', 'javax.swing.JLabel', 'javax.swing.JPanel'], ['java.util.ArrayList', 'java.util.Collections', 'java.util.Comparator', 'java.util.Date', 'java.text.ParseException', 'java.text.SimpleDateFormat'], [], ['java.sql.Connection', 'java.sql.DriverManager', 'java.sql.SQLException', 'java.util.Properties'], ['java.text.NumberFormat', 'java.util.Locale'], ['java.io.BufferedWriter', 'java.io.File', 'java.io.FileWriter', 'javax.swing.JFileChooser'], ['java.text.DateFormat', 'java.text.ParseException', 'java.text.SimpleDateFormat', 'java.util.Calendar', 'java.util.Date', 'java.util.GregorianCalendar'], ['javax.swing.JFrame', 'javax.swing.JTextArea', 'javax.swing.SwingUtilities', 'javax.swing.UIManager'], ['java.lang.reflect.Method'], ['java.util.HashMap'], ['java.lang.reflect.Method', 'java.util.Map', 'java.util.List', 'java.util.HashMap', 'java.util.ArrayList', 'java.util.prefs.Preferences'], ['java.text.ParseException', 'java.text.SimpleDateFormat', 'java.util.Date', 'java.time.format.DateTimeFormatter', 'java.time.temporal.TemporalAccessor'], ['java.lang.reflect.Array', 'java.lang.reflect.Field'], ['java.util.Random', 'java.util.concurrent.ThreadLocalRandom', 'java.util.List', 'java.util.ArrayList', 'java.util.Collection', 'java.util.Iterator', 'java.util.NoSuchElementException'], ['java.io.BufferedInputStream', 'java.io.BufferedOutputStream', 'java.io.DataInputStream', 'java.io.DataOutputStream', 'java.io.File', 'java.io.FileInputStream', 'java.io.FileOutputStream', 'java.io.IOException', 'java.util.ArrayList'], ['java.awt.Component', 'java.awt.event.ActionEvent', 'java.awt.event.ActionListener', 'java.awt.event.InputEvent', 'java.awt.event.KeyEvent', 'java.awt.image.BufferedImage', 'java.io.File', 'javax.imageio.ImageIO', 'javax.swing.ImageIcon', 'javax.swing.JFrame', 'javax.swing.JLabel', 'javax.swing.JMenu', 'javax.swing.JMenuBar', 'javax.swing.JMenuItem', 'javax.swing.JOptionPane', 'javax.swing.JPanel', 'javax.swing.JScrollPane', 'javax.swing.JTextArea', 'javax.swing.JTree', 'javax.swing.KeyStroke', 'javax.swing.SwingUtilities', 'javax.swing.border.TitledBorder'], ['javax.swing.JFrame', 'javax.swing.JProgressBar', 'javax.swing.JLabel', 'java.awt.GridLayout', 'java.awt.Container', 'java.util.List', 'java.beans.PropertyChangeEvent', 'java.beans.PropertyChangeListener', 'javax.swing.SwingWorker', 'java.text.DecimalFormat', 'java.awt.EventQueue'], []]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# Hibernate Zero-shot without Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5"
code_name = "hibernate"
codes = hibernate_codes
predicted_imports = [['org.hibernate.SessionFactory', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.Transaction', 'org.hibernate.cfg.AnnotationConfiguration', 'hibernate.User', 'hibernate.UserDAO'], ['javax.persistence.Entity', 'javax.persistence.CacheConcurrencyStrategy', 'javax.persistence.Id', 'javax.persistence.Column', 'javax.persistence.ManyToOne', 'javax.persistence.OneToMany', 'javax.persistence.JoinColumn', 'java.io.Serializable', 'java.util.List'], ['org.hibernate.dialect.Dialect', 'org.hibernate.dialect.function.StandardSQLFunction', 'org.hibernate.dialect.function.VarArgsSQLFunction', 'org.hibernate.type.Hibernate', 'java.sql.Types'], ['org.hibernate.HibernateException', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration'], ['java.io.File', 'java.net.MalformedURLException', 'java.util.Iterator', 'java.util.List', 'org.dom4j.Document', 'org.dom4j.DocumentException', 'org.dom4j.io.SAXReader', 'org.hibernate.HibernateException', 'org.hibernate.Session', 'org.hibernate.SessionFactory', 'org.hibernate.Transaction', 'org.hibernate.cfg.Configuration', 'org.hibernate.cfg.SchemaExport', 'org.hibernate.engine.EntityMode'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration'], ['java.util.Date', 'javax.persistence.Column', 'javax.persistence.MappedSuperclass', 'javax.persistence.Temporal', 'javax.persistence.TemporalType', 'javax.validation.constraints.AssertTrue', 'javax.validation.constraints.NotNull'], ['java.io.Serializable', 'java.util.List', 'org.hibernate.Session'], ['java.io.Serializable', 'java.util.List', 'org.hibernate.Session'], ['javax.persistence.Entity', 'javax.persistence.Table', 'javax.persistence.Id', 'javax.persistence.Column', 'javax.persistence.ManyToOne', 'javax.persistence.JoinColumn', 'javax.persistence.ForeignKey', 'javax.persistence.FetchType', 'java.io.Serializable'], ['org.hibernate.Session', 'org.hibernate.SessionFactory', 'org.hibernate.Transaction', 'org.hibernate.cfg.Configuration', 'javax.faces.context.FacesContext'], ['javax.persistence.Entity', 'javax.persistence.Table', 'javax.persistence.Id', 'javax.persistence.Column', 'javax.persistence.ManyToOne', 'javax.persistence.JoinColumn', 'javax.persistence.ForeignKey', 'javax.persistence.ManyToMany', 'javax.persistence.JoinTable', 'javax.persistence.CascadeType', 'javax.persistence.FetchType', 'javax.persistence.OneToMany', 'org.hibernate.annotations.IndexColumn', 'java.io.Serializable', 'java.util.Collection', 'java.util.List'], ['javax.persistence.Entity', 'javax.persistence.Table', 'javax.persistence.Id', 'javax.persistence.Column', 'javax.persistence.ManyToOne', 'javax.persistence.JoinColumn', 'javax.persistence.ForeignKey', 'javax.persistence.ManyToMany', 'javax.persistence.CascadeType', 'javax.persistence.FetchType', 'java.io.Serializable', 'java.util.Collection'], ['java.io.ByteArrayOutputStream', 'java.io.IOException', 'java.io.InputStream', 'java.sql.Blob', 'java.sql.SQLException', 'java.util.Date', 'org.hibernate.Hibernate'], ['java.util.List', 'hibernate.Event', 'org.hibernate.Session'], ['org.hibernate.cfg.Configuration', 'org.hibernate.cfg.ConfigurationToShardConfigurationAdapter', 'org.hibernate.shards.ShardConfiguration', 'org.hibernate.shards.ShardId', 'org.hibernate.shards.ShardResolutionStrategy', 'org.hibernate.shards.ShardSelectionStrategy', 'org.hibernate.shards.ShardStrategy', 'org.hibernate.shards.ShardStrategyFactory', 'org.hibernate.shards.access.SequentialShardAccessStrategy', 'org.hibernate.shards.loadbalance.RoundRobinShardLoadBalancer', 'org.hibernate.shards.strategy.AllShardsShardResolutionStrategy', 'org.hibernate.shards.strategy.RoundRobinShardSelectionStrategy', 'org.hibernate.shards.strategy.ShardedConfiguration', 'org.hibernate.shards.strategy.ShardedSessionFactory'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration'], ['javax.persistence.Entity', 'javax.persistence.GeneratedValue', 'javax.persistence.Id', 'javax.persistence.Column', 'org.hibernate.annotations.Table', 'org.hibernate.annotations.Index'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.Session', 'org.hibernate.SessionFactory', 'org.hibernate.Transaction', 'org.hibernate.cfg.Configuration'], ['org.hibernate.Session', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.query.Query', 'com.google.gwt.json.client.JSONObject'], ['org.hibernate.Session', 'org.hibernate.HibernateException', 'org.hibernate.cfg.Configuration', 'org.hibernate.cfg.AnnotationConfiguration'], ['java.lang.reflect.InvocationHandler', 'java.lang.reflect.Method', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration'], ['javax.persistence.Entity', 'javax.persistence.GeneratedValue', 'javax.persistence.Id', 'javax.persistence.OneToOne', 'javax.persistence.CascadeType', 'org.hibernate.annotations.Fetch', 'org.hibernate.annotations.FetchMode'], ['org.hibernate.Session', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.cfg.Configuration', 'org.hibernate.tool.hbm2ddl.SchemaExport', 'org.hibernate.Query', 'org.hibernate.FetchMode', 'org.hibernate.annotations.CascadeType', 'javax.persistence.Entity', 'javax.persistence.GeneratedValue', 'javax.persistence.Id', 'javax.persistence.OneToOne', 'java.util.List'], ['java.util.List', 'org.hibernate.Session', 'org.hibernate.Query', 'hibernate.HibernateUtil'], ['javax.persistence.Entity', 'javax.persistence.GeneratedValue', 'javax.persistence.GenerationType', 'javax.persistence.Id', 'javax.persistence.JoinColumn', 'javax.persistence.ManyToOne', 'javax.persistence.OneToMany', 'javax.persistence.Column', 'org.hibernate.annotations.IndexColumn', 'java.io.Serializable', 'java.util.ArrayList', 'java.util.List'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.Criteria', 'org.hibernate.criterion.Projections', 'java.util.List'], ['javax.persistence.Entity', 'javax.persistence.Table', 'javax.persistence.NamedQueries', 'javax.persistence.NamedQuery', 'javax.persistence.Basic', 'javax.persistence.Column', 'javax.persistence.Id', 'java.io.Serializable'], ['java.util.List', 'org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.cfg.Configuration', 'org.hibernate.query.Query'], ['javax.persistence.Entity', 'javax.persistence.GeneratedValue', 'javax.persistence.Id', 'javax.persistence.Table', 'javax.persistence.Index', 'org.hibernate.annotations.Table'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.cfg.Configuration'], ['org.hibernate.Session', 'org.hibernate.Transaction'], ['org.apache.log4j.BasicConfigurator', 'org.hibernate.Session', 'org.hibernate.Transaction', 'org.hibernate.HibernateException', 'org.hibernate.Query', 'java.util.List', 'java.util.Iterator'], ['org.hibernate.Session', 'org.hibernate.SessionFactory', 'org.hibernate.Transaction', 'org.hibernate.cfg.Configuration'], ['org.hibernate.Session', 'org.hibernate.SessionFactory', 'org.hibernate.HibernateException', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.Transaction', 'org.hibernate.cfg.Configuration'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.Transaction', 'java.io.Serializable', 'java.util.List', 'java.util.Map', 'org.springframework.transaction.annotation.Transactional', 'org.springframework.orm.hibernate5.support.HibernateDaoSupport'], ['org.hibernate.Session', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.query.Query', 'java.util.Iterator', 'java.util.List'], ['org.hibernate.Session', 'org.hibernate.Transaction', 'org.hibernate.Criteria', 'org.hibernate.criterion.Restrictions', 'java.util.List'], ['org.apache.commons.logging.Log', 'org.apache.commons.logging.LogFactory', 'javax.naming.InitialContext', 'org.hibernate.SessionFactory', 'org.hibernate.criterion.Example', 'java.util.List'], ['org.apache.log4j.Logger', 'org.hibernate.Session', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'java.util.Properties'], ['org.apache.struts.actions.DispatchAction', 'org.apache.struts.action.ActionForward', 'org.apache.struts.action.ActionMapping', 'org.apache.struts.action.ActionForm', 'javax.servlet.http.HttpServletRequest', 'javax.servlet.http.HttpServletResponse', 'org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.Transaction', 'org.hibernate.Query', 'java.util.ArrayList', 'java.util.Iterator', 'java.util.HashSet', 'javax.servlet.http.HttpSession'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.HibernateException', 'org.hibernate.JDBCException', 'org.hibernate.CacheMode', 'org.hibernate.Session', 'org.hibernate.SessionFactory'], ['org.hibernate.SessionFactory', 'org.hibernate.Transaction', 'org.hibernate.Session', 'hibernate.HibernateUtil'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.Transaction', 'com.google.gwt.user.server.rpc.RemoteServiceServlet', 'com.google.gwt.user.client.rpc.RemoteService', 'com.google.gwt.user.client.rpc.SerializationException', 'net.sf.gilead.core.hibernate.HibernateUtil', 'net.sf.gilead.gwt.PersistentBeanManager', 'net.sf.gilead.gwt.StatelessProxyStore', 'hibernate.GreetingService'], ['javax.persistence.Entity', 'javax.persistence.Version', 'org.hibernate.annotations.NaturalId', 'java.io.Serializable'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'java.util.List', 'hibernate.Name']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# JodaTime Zero-shot without Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5"
code_name = "jodatime"
codes = jodatime_codes
predicted_imports = [['org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.ISODateTimeFormat', 'java.util.TimeZone'], ['org.joda.time.DateMidnight', 'org.joda.time.DateTime', 'org.joda.time.DateTimeZone'], ['org.joda.time.Interval', 'org.joda.time.PeriodFormatter', 'org.joda.time.PeriodFormatterBuilder'], ['org.joda.time.Period', 'org.joda.time.ReadableInstant'], ['org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'org.joda.time.chrono.GJChronology', 'org.joda.time.chrono.Chronology'], ['org.joda.time.Duration', 'org.joda.time.PeriodBuilder', 'org.joda.time.PeriodFormatterBuilder', 'org.joda.time.PeriodType'], ['java.text.DateFormat', 'java.text.ParseException', 'java.text.SimpleDateFormat', 'java.util.Calendar', 'java.util.Date', 'org.joda.time.DateTime', 'org.joda.time.Days'], ['org.joda.time.Period', 'org.joda.time.PeriodType', 'java.util.concurrent.TimeUnit'], ['java.util.TimeZone', 'org.joda.time.DateTimeZone'], ['java.io.IOException', 'java.io.ObjectInputStream', 'java.io.ObjectOutputStream', 'java.io.Serializable', 'org.joda.time.DateTimeZone', 'org.joda.time.chrono.ISOChronology'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'java.text.ParseException', 'java.text.SimpleDateFormat'], ['java.text.ParseException', 'java.text.SimpleDateFormat', 'java.util.Date', 'org.joda.time.DateTime'], ['org.junit.Test', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatter', 'org.joda.time.format.PeriodFormatterBuilder'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.MutableDateTime', 'org.joda.time.DateTime'], ['org.joda.time.DateTime', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatter', 'org.joda.time.format.PeriodFormatterBuilder'], ['org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.DateTime'], ['org.joda.time.DateTimeZone', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormatterBuilder'], ['org.joda.time.Instant', 'org.joda.time.Interval', 'org.joda.time.ReadableInstant', 'java.util.Date'], ['org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter'], ['java.text.DateFormat', 'java.text.ParseException', 'java.text.SimpleDateFormat', 'java.util.Date', 'org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'org.joda.time.LocalDateTime'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter', 'java.text.ParseException', 'java.util.Date'], ['org.joda.time.LocalDate', 'org.joda.time.DateTimeConstants'], ['org.joda.time.DateTime', 'org.joda.time.Period', 'org.joda.time.PeriodType', 'org.joda.time.format.PeriodFormat'], ['org.joda.time.LocalDateTime', 'org.joda.time.Days', 'org.joda.time.DateMidnight'], ['org.joda.time.Duration', 'org.joda.time.Period'], ['java.util.ArrayList', 'org.joda.time.DateTime'], ['org.joda.time.DateTimeFormatter', 'org.joda.time.DateTime', 'org.joda.time.LocalDate', 'org.joda.time.Days', 'org.joda.time.format.DateTimeFormat', 'java.util.ArrayList', 'java.util.List'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeParser'], ['org.joda.time.DateTimeZone', 'org.joda.time.DateTime'], ['org.joda.time.DateTime', 'org.joda.time.Weeks'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.Period', 'org.joda.time.format.PeriodFormatter', 'org.joda.time.format.PeriodFormatterBuilder'], ['org.joda.time.Period', 'org.joda.time.format.PeriodFormat', 'org.joda.time.format.PeriodFormatter', 'java.util.Locale'], ['org.joda.time.DateTime', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatter', 'org.joda.time.format.PeriodFormatterBuilder'], ['org.joda.time.Period', 'org.joda.time.format.PeriodFormatter', 'org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.format.ISOPeriodFormat'], ['org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'java.util.concurrent.TimeUnit'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.DateTime'], ['org.joda.time.Date', 'org.joda.time.DateTime', 'org.joda.time.Days'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.LocalDate'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormatterBuilder', 'org.joda.time.format.DateTimeParser'], ['org.joda.time.Period', 'org.joda.time.format.PeriodFormatter', 'org.joda.time.format.PeriodFormatterBuilder'], ['java.util.Set', 'java.util.HashSet', 'java.util.TimeZone', 'java.util.Date', 'org.joda.time.DateTimeZone', 'org.joda.time.tz.Provider'], ['org.joda.time.DateTime', 'org.joda.time.DateTimeZone'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.DateTime', 'org.joda.time.MillisProvider'], ['org.joda.time.LocalDate', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.DateTime']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# GWT Zero-shot wihtout Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5"
code_name = "gwt"
codes = gwt_codes
predicted_imports = [['com.google.gwt.user.client.ui.Composite', 'com.google.gwt.event.dom.client.MouseDownHandler', 'com.google.gwt.event.dom.client.MouseDownEvent', 'com.google.gwt.user.client.ui.AbsolutePanel'], ['com.google.gwt.junit.client.GWTTestCase', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.user.client.rpc.GWT', 'com.google.gwt.sample.stockwatcher.client.GreetingService', 'com.google.gwt.sample.stockwatcher.client.GreetingServiceAsync'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.Window', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.TextBox'], ['com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.VerticalSplitPanel', 'com.google.gwt.user.client.ui.HTML'], ['com.google.gwt.core.client.JsArray', 'com.google.gwt.core.client.JsArrayInteger'], ['com.google.gwt.user.client.DOM', 'com.google.gwt.user.client.Element', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.Timer', 'com.google.gwt.user.client.ui.Panel', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.VerticalPanel'], ['com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Response', 'com.google.gwt.user.client.Window', 'com.google.gwt.user.client.rpc.RequestException'], ['com.google.gwt.user.client.Command', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.MenuBar'], ['com.google.gwt.user.client.ui.FormPanel', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickListener', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.RootPanel'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.VerticalPanel'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.ClickListener', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.core.client.GWT', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Response', 'com.google.gwt.user.client.Window'], ['com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.URL', 'com.google.gwt.http.client.RequestException'], ['com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.HTML'], ['com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.Image', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.DockPanel', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.ImageBundle', 'com.google.gwt.user.client.ui.AbstractImagePrototype'], ['com.google.gwt.core.client.GWT', 'com.google.gwt.uibinder.client.UiBinder', 'com.google.gwt.uibinder.client.UiBinder.Widget', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.RootPanel'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.Window', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.RootPanel'], ['com.google.gwt.user.client.ui.PopupPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.VerticalPanel'], ['com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.uibinder.client.UiBinder', 'com.google.gwt.uibinder.client.UiField', 'com.google.gwt.uibinder.client.UiHandler', 'com.google.gwt.core.client.GWT'], ['java.util.UUID', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response'], ['com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.user.client.DOM', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.event.dom.client.HasClickHandlers'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.KeyCodes', 'com.google.gwt.event.dom.client.KeyUpEvent', 'com.google.gwt.event.dom.client.KeyUpHandler', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.DialogBox', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.VerticalPanel'], ['com.google.gwt.user.client.ui.FormPanel', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickListener', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.FormHandler', 'com.google.gwt.event.dom.client.FormSubmitEvent', 'com.google.gwt.event.dom.client.FormSubmitCompleteEvent', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.core.client.GWT', 'com.google.gwt.dom.client.ButtonElement', 'com.google.gwt.dom.client.Document', 'com.google.gwt.dom.client.NodeList', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.RootPanel'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Response', 'com.google.gwt.user.client.Window'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.core.client.JsonpRequestBuilder', 'com.google.gwt.user.client.Window', 'com.google.gwt.user.client.rpc.AsyncCallback'], ['com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.DialogBox', 'com.google.gwt.user.client.ui.Grid', 'com.google.gwt.user.client.ui.HTML'], ['com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.URL'], ['com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.user.client.Window', 'com.google.gwt.http.client.URL'], ['java.util.ArrayList', 'java.util.Iterator', 'java.util.List', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.CheckBox', 'com.google.gwt.user.client.ui.ClickListener', 'com.google.gwt.user.client.ui.DockPanel', 'com.google.gwt.user.client.ui.Grid', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.RadioButton', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.dom.client.DOM', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.SubmitEvent', 'com.google.gwt.event.dom.client.SubmitHandler', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.FormPanel'], ['com.google.gwt.animation.client.Animation', 'com.google.gwt.core.client.GWT', 'com.google.gwt.dom.client.Element', 'com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.user.client.ui.FlowPanel'], ['com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.user.client.ui.SimplePanel', 'com.google.gwt.user.client.ui.RootPanel'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Response', 'com.google.gwt.user.client.Window', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.RootPanel'], ['com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.Hyperlink', 'com.google.gwt.event.dom.client.HasClickHandlers'], ['java.io.DataInputStream', 'java.io.File', 'java.io.FileInputStream', 'javax.servlet.ServletContext', 'javax.servlet.ServletOutputStream', 'javax.servlet.http.HttpServletResponse', 'com.google.gwt.user.server.rpc.RemoteServiceServlet'], ['com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.Label', 'com.sencha.gxt.widget.core.client.container.LayoutContainer'], ['com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.Element', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.ClickEvent'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Response', 'com.google.gwt.user.client.Window', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.MouseOutEvent', 'com.google.gwt.event.dom.client.MouseOutHandler', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.core.client.GWT', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.user.client.ui.ResizeComposite', 'com.google.gwt.user.client.ui.DockLayoutPanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.core.client.RunAsyncCallback'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.Window', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.RootPanel'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.DialogBox', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.RootPanel'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.LayoutPanel', 'com.google.gwt.user.client.ui.RootLayoutPanel', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.event.logical.shared.BeforeSelectionEvent', 'com.google.gwt.event.logical.shared.BeforeSelectionHandler', 'com.google.gwt.event.logical.shared.SelectionEvent', 'com.google.gwt.event.logical.shared.SelectionHandler', 'com.google.gwt.user.client.Window', 'com.google.gwt.user.client.ui.TabBar'], ['java.util.ArrayList', 'com.google.gwt.dom.client.BodyElement', 'com.google.gwt.dom.client.Document', 'com.google.gwt.dom.client.Element', 'com.google.gwt.dom.client.DOM'], ['com.google.gwt.user.client.ui.ComplexPanel', 'com.google.gwt.user.client.ui.HasText', 'com.google.gwt.user.client.ui.HasHTML', 'com.google.gwt.user.client.ui.HasClickHandlers', 'com.google.gwt.user.client.ui.HasKeyDownHandlers', 'com.google.gwt.user.client.ui.HasBlurHandlers', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.KeyDownHandler', 'com.google.gwt.event.dom.client.BlurHandler', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.KeyDownEvent', 'com.google.gwt.event.dom.client.BlurEvent', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.DOM']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# XStream Zero-shot without Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5"
code_name = "xstream"
codes = xstream_codes
predicted_imports = [['com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['java.io.BufferedReader', 'java.io.FileReader', 'java.io.IOException', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.json.JettisonMappedXmlDriver', 'java.util.ArrayList', 'java.util.List'], ['java.util.ArrayList', 'com.thoughtworks.xstream.XStream'], ['java.io.FileNotFoundException', 'java.io.PrintWriter', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['java.io.File', 'java.io.FileOutputStream', 'java.io.OutputStream', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['com.thoughtworks.xstream.annotations.XStreamAsAttribute'], ['android.app.Activity', 'android.os.Bundle', 'android.widget.TextView', 'org.ksoap2.SoapEnvelope', 'org.ksoap2.serialization.SoapObject', 'org.ksoap2.serialization.SoapSerializationEnvelope', 'org.ksoap2.transport.HttpTransportSE', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['java.io.FileInputStream', 'java.io.FileNotFoundException', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['java.io.StringWriter', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.PrettyPrintWriter'], ['java.util.ArrayList', 'java.util.List', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.converters.collections.CollectionConverter', 'com.thoughtworks.xstream.mapper.ClassAliasingMapper'], ['java.util.List', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.mapper.ClassAliasingMapper', 'com.thoughtworks.xstream.mapper.Mapper'], ['com.thoughtworks.xstream.converters.SingleValueConverter', 'com.thoughtworks.xstream.converters.enums.AbstractSingleValueConverter'], ['java.util.HashSet', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.converters.collections.CollectionConverter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.MarshallingContext'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.mapper.MapperWrapper'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.io.xml.XppDriver', 'com.thoughtworks.xstream.io.xml.xppdom.XppFactory', 'com.thoughtworks.xstream.io.xml.xppdom.XppReader', 'com.thoughtworks.xstream.io.xml.xppdom.XppWriter', 'com.thoughtworks.xstream.io.xml.xppdom.XppDomDriver', 'com.thoughtworks.xstream.io.xml.xppdom.XppDom', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['java.util.Map', 'java.util.HashMap', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['java.io.File', 'java.io.FileInputStream', 'java.io.InputStreamReader', 'java.io.InputStream', 'java.io.Reader', 'java.nio.charset.Charset', 'com.thoughtworks.xstream.XStream'], ['com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext', 'com.thoughtworks.xstream.converters.reflection.ReflectionConverter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.mapper.Mapper.Null', 'com.thoughtworks.xstream.mapper.MapperWrapper', 'com.thoughtworks.xstream.mapper.ReflectionProvider', 'com.thoughtworks.xstream.mapper.StandardMapper', 'java.util.Map'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.annotations.XStreamConverter', 'com.thoughtworks.xstream.annotations.XStreamImplicit', 'com.thoughtworks.xstream.converters.extended.ToAttributedValueConverter', 'java.io.File', 'java.util.Arrays', 'java.util.List'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.converters.extended.JavaBeanConverter', 'com.thoughtworks.xstream.io.xml.XppDriver'], ['com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.mapper.ReflectionProvider'], ['com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.annotations.XStreamConverter', 'com.thoughtworks.xstream.converters.extended.ToAttributedValueConverter'], ['java.util.ArrayList', 'java.util.Collection', 'java.util.List', 'com.thoughtworks.xstream.converters.SingleValueConverter'], ['org.apache.commons.lang3.text.WordUtils', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext', 'xstream.Content'], ['java.util.Map', 'java.util.HashMap', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['java.util.ArrayList', 'java.util.List', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.MarshallingContext', 'com.thoughtworks.xstream.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.mapper.MapperWrapper', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.converters.collections.CollectionConverter', 'com.thoughtworks.xstream.converters.collections.MapConverter', 'com.thoughtworks.xstream.converters.collections.SortedMapConverter', 'com.thoughtworks.xstream.converters.collections.SortedSetConverter'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver', 'com.thoughtworks.xstream.mapper.Mapper', 'xstream.ActionMessage', 'xstream.MessageHeader', 'xstream.MessageBody', 'xstream.RemotingMessage', 'xstream.CommandMessage', 'xstream.AcknowledgeMessage', 'xstream.ErrorMessage', 'xstream.ASObject', 'xstream.AsyncMessage', 'xstream.CommandMessageExt', 'xstream.AcknowledgeMessageExt'], ['java.util.ArrayList', 'java.util.Iterator', 'java.util.List', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.annotations.XStreamImplicit', 'com.thoughtworks.xstream.mapper.MapperWrapper'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.converters.SingleValueConverter', 'com.thoughtworks.xstream.core.mapper.Mapper', 'com.thoughtworks.xstream.core.util.MapperWrapper', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.io.xml.CompactWriter', 'com.thoughtworks.xstream.io.xml.XppDriver', 'com.thoughtworks.xstream.mapper.MapperWrapper', 'com.thoughtworks.xstream.reflect.ReflectionProvider'], ['java.util.Map', 'java.util.HashMap', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.StaxDriver'], ['java.util.ArrayList', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.io.MarshallingContext', 'com.thoughtworks.xstream.io.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'java.util.ArrayList', 'java.util.List'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.io.Writer', 'com.thoughtworks.xstream.io.xml.PrettyPrintWriter', 'com.thoughtworks.xstream.io.xml.XppDriver'], ['com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['java.util.List', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.StaxDriver'], ['com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.annotations.XStreamImplicit', 'java.util.List'], ['java.util.List', 'java.util.ArrayList', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['java.io.File', 'java.io.FileOutputStream', 'com.thoughtworks.xstream.XStream']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

### With Self-consistency Temp 1

#### From 10 Sample

In [ ]:
# Android Zero-shot Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot/android"
code_name = "android"
codes = android_codes
predicted_imports = [['android.os.Bundle', 'android.app.Activity', 'android.widget.TextView'], ['android.os.Bundle', 'android.app.Activity'], ['com.google.android.maps.ItemizedOverlay', 'com.google.android.maps.GeoPoint', 'android.graphics.drawable.Drawable', 'com.google.android.maps.OverlayItem'], ['android.os.Bundle', 'android.app.Activity', 'android.widget.TextView'], ['android.graphics.drawable.Drawable', 'android.widget.TabHost.TabSpec', 'android.widget.TabHost'], ['android.widget.LinearLayout', 'android.view.View', 'android.view.Gravity', 'android.view.ViewGroup'], ['android.widget.ArrayAdapter', 'android.os.Bundle', 'android.view.View', 'android.app.Activity'], ['java.io.IOException', 'java.net.UnknownHostException', 'java.net.Socket', 'java.net.InetAddress', 'android.util.Log', 'java.net.Inet4Address'], ['java.util.TimerTask', 'android.os.Bundle', 'java.util.Timer', 'android.widget.TextView'], ['android.os.SystemClock', 'android.widget.Chronometer', 'android.util.Log'], ['com.google.android.maps.GeoPoint', 'android.view.MotionEvent', 'android.content.Context', 'com.google.android.maps.MyLocationOverlay', 'android.graphics.Canvas', 'android.location.Location', 'com.google.android.maps.MapView'], ['android.widget.LinearLayout', 'android.app.Activity', 'android.os.Bundle', 'android.graphics.Color', 'android.view.ViewGroup', 'android.widget.TextView'], ['java.io.ByteArrayOutputStream', 'java.io.FileNotFoundException', 'java.io.IOException', 'android.net.Uri', 'android.content.ContentResolver', 'java.io.ByteArrayInputStream', 'android.graphics.Bitmap', 'android.provider.MediaStore.Images.Media'], ['java.io.FileNotFoundException', 'java.io.IOException', 'java.net.MalformedURLException', 'java.net.ProtocolException', 'java.net.HttpURLConnection', 'java.io.InputStream', 'java.io.File', 'java.io.FileOutputStream', 'java.net.URL'], ['android.net.Uri', 'android.content.Intent'], ['android.util.Log', 'android.content.Context', 'android.view.MotionEvent', 'android.util.AttributeSet', 'android.view.View', 'android.graphics.Canvas', 'android.widget.SeekBar'], ['java.io.IOException', 'java.io.InputStreamReader', 'android.util.Log', 'org.apache.http.client.methods.HttpGet', 'org.apache.http.impl.client.DefaultHttpClient', 'org.apache.http.client.HttpClient', 'org.apache.http.HttpEntity', 'org.apache.http.HttpResponse', 'java.net.URISyntaxException', 'java.net.URI'], ['android.view.View.OnClickListener', 'android.app.Activity', 'android.os.Bundle', 'android.content.DialogInterface', 'android.content.Intent'], ['android.widget.ImageView', 'android.content.Context', 'android.view.View', 'android.widget.GridView', 'android.view.ViewGroup'], ['android.view.View', 'android.view.animation.ScaleAnimation', 'android.content.Context', 'android.widget.TextView'], ['androidExamples.R', 'android.widget.LinearLayout', 'android.widget.ImageButton', 'android.content.Context'], ['android.widget.ListView', 'android.database.Cursor', 'android.widget.SimpleCursorAdapter', 'android.os.Bundle', 'android.view.View', 'android.app.ListActivity', 'android.widget.ListAdapter', 'android.provider.Contacts.People', 'android.content.Intent', 'android.widget.TextView'], ['java.util.Random', 'java.io.IOException', 'android.widget.ListView', 'java.net.MalformedURLException', 'android.app.Activity', 'android.widget.ImageView', 'android.os.Bundle', 'java.net.HttpURLConnection', 'android.graphics.Bitmap', 'java.util.ArrayList', 'android.graphics.BitmapFactory', 'android.widget.ArrayAdapter', 'java.net.URL'], ['java.util.Iterator', 'android.app.Activity', 'java.util.List', 'android.os.Bundle', 'android.util.Log', 'android.content.Context', 'android.location.LocationManager'], ['android.os.Bundle', 'android.database.sqlite.SQLiteDatabase', 'android.content.Context', 'java.util.ArrayList', 'android.app.ListActivity'], ['android.widget.ScrollView', 'android.view.MotionEvent', 'android.content.Context', 'android.view.View', 'android.view.View.OnTouchListener', 'android.view.KeyEvent'], ['android.widget.BaseAdapter', 'android.content.Context', 'android.view.View', 'android.widget.GridView', 'android.view.ViewGroup', 'android.widget.TextView'], ['android.app.Application', 'android.content.Context'], ['android.app.Service', 'android.os.IBinder', 'android.content.Context', 'android.os.PowerManager', 'android.content.Intent'], ['org.apache.http.util.EntityUtils', 'org.apache.http.client.methods.HttpGet', 'org.apache.http.client.HttpClient', 'org.apache.http.impl.client.DefaultHttpClient', 'org.apache.http.HttpEntity', 'org.apache.http.HttpResponse', 'org.apache.http.HttpHost'], ['android.os.PowerManager', 'android.content.Context'], ['android.os.Environment', 'java.io.IOException', 'java.io.File', 'android.media.MediaRecorder'], ['java.io.IOException', 'android.view.SurfaceHolder', 'android.content.Context', 'android.hardware.Camera', 'android.graphics.PixelFormat', 'android.hardware.Camera.Parameters'], ['android.view.View.OnClickListener', 'android.widget.ListView', 'android.widget.TabHost', 'android.widget.TabHost.TabContentFactory', 'android.os.Bundle', 'android.util.Log', 'android.widget.TabHost.OnTabChangeListener', 'android.view.View', 'android.app.TabActivity'], ['java.io.IOException', 'java.io.File', 'android.media.MediaRecorder'], ['android.content.ContentValues', 'android.database.sqlite.SQLiteDatabase'], ['android.os.Handler', 'android.app.ProgressDialog', 'android.app.Activity', 'android.os.Message'], ['java.io.DataOutputStream', 'java.io.IOException', 'java.net.MalformedURLException', 'java.net.HttpURLConnection', 'java.io.InputStream', 'java.io.FileInputStream', 'java.io.File', 'java.net.URL', 'java.io.DataInputStream'], ['android.widget.LinearLayout', 'com.google.android.maps.MapActivity', 'android.os.Bundle', 'com.google.android.maps.MyLocationOverlay', 'android.view.View', 'com.google.android.maps.MapController', 'com.google.android.maps.MapView', 'com.google.android.maps.Overlay'], ['android.view.View.OnClickListener', 'android.app.Activity', 'android.widget.ImageView', 'android.os.Bundle', 'android.util.Log', 'android.widget.SlidingDrawer', 'android.view.View'], ['android.graphics.Color', 'android.view.View', 'android.content.Context', 'android.widget.TextView'], ['android.media.MediaPlayer.OnPreparedListener', 'android.media.MediaPlayer.OnErrorListener', 'android.view.SurfaceView', 'android.media.MediaPlayer.OnBufferingUpdateListener', 'android.os.Bundle', 'java.io.InputStream', 'android.view.View', 'java.io.File', 'java.net.URLConnection', 'java.net.URL', 'java.io.IOException', 'android.webkit.URLUtil', 'android.util.Log', 'android.media.MediaPlayer', 'android.media.MediaPlayer.OnCompletionListener', 'java.io.FileOutputStream', 'android.view.SurfaceHolder', 'android.app.Activity', 'android.graphics.PixelFormat', 'android.widget.Button', 'android.content.pm.ActivityInfo', 'android.content.Intent'], ['org.apache.http.util.EntityUtils', 'java.io.IOException', 'org.apache.http.client.methods.HttpGet', 'org.apache.http.client.HttpClient', 'org.apache.http.impl.client.DefaultHttpClient', 'org.apache.http.HttpEntity', 'org.apache.http.HttpResponse', 'org.apache.http.HttpHost'], ['android.app.Service', 'android.os.IBinder', 'android.content.Intent'], ['android.content.ComponentName', 'android.app.Activity', 'android.content.IntentFilter', 'android.os.Bundle', 'android.util.Log', 'android.content.Context', 'android.content.BroadcastReceiver', 'android.webkit.MalformedMimeTypeException', 'android.content.Intent'], ['android.app.Activity', 'android.os.Bundle', 'android.widget.EditText', 'android.view.View', 'android.widget.Button', 'android.content.Intent'], ['java.io.IOException', 'android.app.Activity', 'android.os.Bundle', 'org.apache.http.client.ClientProtocolException', 'org.apache.http.client.methods.HttpGet', 'org.apache.http.impl.client.DefaultHttpClient', 'android.widget.TextView'], ['android.widget.ImageView', 'android.widget.TextView', 'android.hardware.Camera.PictureCallback', 'android.content.Context', 'android.hardware.Camera', 'android.graphics.Bitmap', 'android.graphics.BitmapFactory', 'android.app.AlertDialog', 'android.app.Dialog'], ['android.widget.LinearLayout', 'com.example.R', 'android.view.Gravity', 'android.content.Context', 'android.util.AttributeSet', 'android.view.LayoutInflater'], ['android.app.Activity', 'android.os.Bundle', 'android.widget.EditText', 'android.util.Log', 'android.view.View', 'android.widget.Button', 'java.text.NumberFormat', 'android.widget.TextView']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# JDK Zero-shot Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot/jdk"
code_name = "jdk"
codes = jdk_codes
predicted_imports = [['java.applet.Applet', 'java.io.ObjectInputStream', 'java.io.IOException', 'java.io.ObjectOutputStream', 'java.awt.Graphics', 'java.awt.Frame', 'java.awt.Color', 'java.awt.Dimension', 'java.awt.event.WindowAdapter', 'java.io.FileInputStream', 'java.awt.Graphics2D', 'java.awt.image.BufferedImage', 'java.io.FileOutputStream', 'java.awt.Container', 'java.awt.event.WindowEvent'], ['javax.swing.JCheckBox', 'java.awt.BorderLayout', 'java.awt.GridLayout', 'java.awt.GridBagLayout', 'java.awt.image.BufferedImage', 'java.awt.Dimension', 'javax.swing.JButton', 'java.awt.Graphics2D', 'javax.swing.JTable', 'java.awt.GradientPaint', 'javax.swing.JScrollPane', 'javax.swing.JComboBox', 'javax.swing.JSplitPane', 'javax.swing.SwingUtilities', 'java.awt.Color', 'javax.swing.table.DefaultTableModel', 'java.awt.event.ActionEvent', 'javax.swing.border.TitledBorder', 'java.awt.FlowLayout', 'java.awt.event.ActionListener', 'javax.swing.JPanel', 'javax.swing.ImageIcon', 'javax.swing.JLabel', 'javax.swing.JFrame', 'javax.swing.UIManager'], ['java.math.RoundingMode', 'java.text.DecimalFormatSymbols', 'java.text.ParseException', 'java.text.DecimalFormat', 'java.util.Locale'], ['java.util.ArrayList', 'java.util.regex.Matcher', 'java.util.List', 'java.util.regex.Pattern'], ['java.util.LinkedHashMap', 'java.util.Collections', 'java.util.List', 'java.util.LinkedList', 'java.util.Comparator', 'java.util.HashMap', 'java.util.TreeMap', 'java.util.Map'], ['java.util.Random', 'javax.swing.JFrame', 'java.util.Observable', 'java.awt.event.ActionEvent', 'java.awt.Component', 'java.awt.Graphics', 'javax.swing.JLabel', 'java.util.Observer', 'java.awt.event.ActionListener', 'javax.swing.JPanel', 'javax.swing.JButton', 'java.awt.Graphics2D', 'java.awt.RenderingHints', 'java.awt.EventQueue', 'java.awt.BorderLayout', 'java.awt.Color', 'javax.swing.Icon'], ['java.util.Collections', 'java.text.ParseException', 'java.util.ArrayList', 'java.util.Comparator', 'java.util.Date', 'java.text.SimpleDateFormat'], [], ['java.sql.Connection', 'java.sql.SQLException', 'java.util.Properties', 'java.sql.DriverManager'], ['java.text.NumberFormat', 'java.util.Locale'], ['javax.swing.JFileChooser', 'java.io.BufferedWriter', 'java.io.File', 'java.io.FileWriter'], ['java.text.DateFormat', 'java.util.Calendar', 'java.text.ParseException', 'java.util.GregorianCalendar', 'java.util.Date', 'java.text.SimpleDateFormat'], ['javax.swing.SwingUtilities', 'javax.swing.JFrame', 'javax.swing.JTextArea', 'javax.swing.UIManager'], ['java.lang.Class', 'java.lang.reflect.Method'], ['java.util.HashMap'], ['java.lang.reflect.Method', 'java.util.List', 'java.lang.reflect.InvocationTargetException', 'java.util.ArrayList', 'java.util.prefs.Preferences', 'java.util.HashMap', 'java.util.Map'], ['java.time.temporal.TemporalAccessor', 'java.text.ParseException', 'java.util.Date', 'java.text.SimpleDateFormat', 'java.time.format.DateTimeFormatter'], ['java.lang.reflect.Field', 'java.lang.reflect.Array'], ['java.util.Random', 'java.util.Iterator', 'java.util.List', 'java.util.concurrent.ThreadLocalRandom', 'java.util.ArrayList', 'java.util.Collection', 'java.util.NoSuchElementException'], ['java.io.DataOutputStream', 'java.io.IOException', 'java.io.BufferedInputStream', 'java.io.BufferedOutputStream', 'java.util.ArrayList', 'java.io.FileInputStream', 'java.io.File', 'java.io.FileOutputStream', 'java.io.DataInputStream'], ['javax.swing.JMenuBar', 'java.io.File', 'javax.swing.JTextArea', 'java.awt.BorderLayout', 'javax.swing.KeyStroke', 'java.awt.event.KeyEvent', 'java.awt.image.BufferedImage', 'javax.swing.JTree', 'javax.swing.JOptionPane', 'javax.swing.JLabel', 'java.awt.event.InputEvent', 'javax.imageio.ImageIO', 'javax.swing.JScrollPane', 'javax.swing.JMenu', 'javax.swing.SwingUtilities', 'java.awt.event.ActionEvent', 'java.awt.Component', 'javax.swing.border.TitledBorder', 'java.awt.Image', 'java.awt.event.ActionListener', 'javax.swing.JPanel', 'javax.swing.ImageIcon', 'javax.swing.JMenuItem', 'javax.swing.JFrame'], ['javax.swing.JFrame', 'java.util.List', 'javax.swing.SwingWorker', 'java.beans.PropertyChangeEvent', 'java.text.DecimalFormat', 'javax.swing.JLabel', 'java.awt.EventQueue', 'java.beans.PropertyChangeListener', 'javax.swing.JProgressBar', 'java.awt.Container', 'java.awt.GridLayout'], []]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# Hibernate Zero-shot Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot/hibernate"
code_name = "hibernate"
codes = hibernate_codes
predicted_imports = [['org.hibernate.SessionFactory', 'org.hibernate.cfg.AnnotationConfiguration'], ['hibernate.User', 'org.hibernate.SessionFactory', 'hibernate.UserDAO', 'org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.Session', 'org.hibernate.Transaction'], ['java.io.Serializable', 'javax.persistence.Id', 'java.util.List', 'javax.persistence.JoinColumn', 'javax.persistence.ManyToOne', 'org.hibernate.annotations.CacheConcurrencyStrategy', 'javax.persistence.Column', 'javax.persistence.Cacheable', 'javax.persistence.OneToMany', 'javax.persistence.Entity', 'org.hibernate.annotations.Cache'], ['org.hibernate.dialect.Dialect', 'org.hibernate.dialect.function.StandardSQLFunction', 'org.hibernate.dialect.function.VarArgsSQLFunction', 'org.hibernate.dialect.function.SQLFunctionTemplate', 'org.hibernate.Hibernate', 'java.sql.Types'], ['org.hibernate.SessionFactory', 'org.hibernate.HibernateException', 'org.hibernate.cfg.Configuration'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration'], ['org.dom4j.EntityMode', 'java.util.Iterator', 'java.net.MalformedURLException', 'org.hibernate.HibernateException', 'org.hibernate.tool.hbm2ddl.SchemaExport', 'java.util.List', 'org.hibernate.SessionFactory', 'org.dom4j.io.SAXReader', 'java.io.File', 'org.hibernate.cfg.Configuration', 'org.dom4j.DocumentException', 'org.dom4j.Document', 'org.hibernate.Session', 'org.hibernate.Transaction'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.Session'], ['javax.validation.constraints.AssertTrue', 'javax.persistence.Temporal', 'javax.validation.constraints.NotNull', 'javax.persistence.MappedSuperclass', 'javax.persistence.Column', 'java.util.Date', 'javax.persistence.Transient', 'javax.persistence.TemporalType'], ['java.util.List', 'java.io.Serializable', 'org.hibernate.Session'], ['java.util.List', 'java.io.Serializable', 'org.hibernate.Session'], ['javax.persistence.Id', 'java.io.Serializable', 'javax.persistence.ForeignKey', 'javax.persistence.ManyToOne', 'javax.persistence.JoinColumn', 'javax.persistence.Column', 'javax.persistence.FetchType', 'javax.persistence.Entity', 'javax.persistence.Table'], ['org.hibernate.SessionFactory', 'javax.faces.context.FacesContext', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session', 'org.hibernate.Transaction'], ['java.io.Serializable', 'javax.persistence.Id', 'javax.persistence.IndexColumn', 'javax.persistence.ManyToMany', 'javax.persistence.ForeignKey', 'java.util.List', 'javax.persistence.JoinColumn', 'javax.persistence.ManyToOne', 'javax.persistence.CascadeType', 'javax.persistence.Column', 'javax.persistence.OneToMany', 'javax.persistence.FetchType', 'java.util.Collection', 'javax.persistence.Entity', 'javax.persistence.JoinTable', 'javax.persistence.Table'], ['javax.persistence.Id', 'javax.persistence.ManyToMany', 'java.io.Serializable', 'javax.persistence.ForeignKey', 'javax.persistence.ManyToOne', 'javax.persistence.JoinColumn', 'javax.persistence.CascadeType', 'javax.persistence.Column', 'javax.persistence.FetchType', 'java.util.Collection', 'javax.persistence.Entity', 'javax.persistence.Table'], ['java.sql.Blob', 'java.io.ByteArrayOutputStream', 'java.io.IOException', 'java.sql.SQLException', 'java.io.InputStream', 'java.util.Date', 'org.hibernate.Hibernate'], ['java.util.List', 'hibernate.Event', 'org.hibernate.Session'], [], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration'], ['javax.persistence.Id', 'org.hibernate.annotations.Table', 'javax.persistence.Index', 'javax.persistence.Column', 'javax.persistence.Entity', 'javax.persistence.GeneratedValue'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session', 'org.hibernate.Transaction'], ['org.hibernate.SessionFactory', 'org.hibernate.query.Query', 'com.google.gwt.json.client.JSONObject', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session'], ['org.hibernate.HibernateException', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.Session'], ['org.hibernate.SessionFactory', 'java.lang.reflect.InvocationHandler', 'java.lang.reflect.Method', 'org.hibernate.cfg.Configuration'], ['javax.persistence.Id', 'javax.persistence.CascadeType', 'javax.persistence.OneToOne', 'javax.persistence.Entity', 'org.hibernate.annotations.FetchMode', 'javax.persistence.GeneratedValue', 'org.hibernate.annotations.Fetch'], ['javax.persistence.Id', 'org.hibernate.tool.hbm2ddl.SchemaExport', 'java.util.List', 'org.hibernate.SessionFactory', 'org.hibernate.annotations.CascadeType', 'javax.persistence.OneToOne', 'javax.persistence.Entity', 'org.hibernate.Query', 'org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.annotations.FetchMode', 'javax.persistence.GeneratedValue', 'org.hibernate.Session', 'org.hibernate.annotations.Fetch'], ['java.util.List', 'org.hibernate.Session', 'org.hibernate.Query'], ['java.io.Serializable', 'javax.persistence.Id', 'javax.persistence.IndexColumn', 'java.util.List', 'javax.persistence.JoinColumn', 'javax.persistence.ManyToOne', 'java.util.ArrayList', 'javax.persistence.Column', 'javax.persistence.OneToMany', 'javax.persistence.GenerationType', 'javax.persistence.Entity', 'javax.persistence.GeneratedValue'], ['org.hibernate.SessionFactory', 'java.util.List', 'org.hibernate.criterion.Projections', 'org.hibernate.Session', 'org.hibernate.Criteria'], ['javax.persistence.Id', 'java.io.Serializable', 'javax.persistence.NamedQuery', 'javax.persistence.Basic', 'javax.persistence.NamedQueries', 'javax.persistence.Column', 'javax.persistence.Entity', 'javax.persistence.Table'], ['java.util.List', 'org.hibernate.SessionFactory', 'org.hibernate.query.Query', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session'], ['javax.persistence.Id', 'javax.persistence.Index', 'javax.persistence.Entity', 'javax.persistence.GeneratedValue', 'javax.persistence.Table'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.cfg.Configuration'], ['org.hibernate.Session', 'org.hibernate.Transaction'], ['org.hibernate.HibernateException', 'java.util.Iterator', 'java.util.List', 'org.apache.log4j.BasicConfigurator', 'org.hibernate.Query', 'org.hibernate.Session', 'org.hibernate.Transaction'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session', 'org.hibernate.Transaction'], ['org.hibernate.SessionFactory', 'org.hibernate.HibernateException', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session', 'org.hibernate.Transaction'], ['java.io.Serializable', 'java.util.List', 'org.hibernate.SessionFactory', 'org.springframework.transaction.annotation.Transactional', 'org.hibernate.Session', 'java.util.Map', 'org.hibernate.Transaction'], ['java.util.Iterator', 'java.util.List', 'org.hibernate.SessionFactory', 'org.hibernate.query.Query', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session'], ['org.hibernate.Criteria', 'org.hibernate.Transaction', 'java.util.List', 'org.hibernate.Session', 'org.hibernate.criterion.Restrictions'], ['java.util.List', 'org.hibernate.SessionFactory', 'javax.naming.InitialContext', 'org.hibernate.criterion.Example', 'org.apache.commons.logging.Log', 'org.apache.commons.logging.LogFactory'], ['org.hibernate.SessionFactory', 'java.util.Properties', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session', 'org.apache.log4j.Logger'], ['javax.servlet.http.HttpSession', 'java.util.Iterator', 'java.util.Set', 'org.apache.struts.actions.DispatchAction', 'javax.servlet.http.HttpServletRequest', 'org.hibernate.Transaction', 'org.hibernate.SessionFactory', 'org.apache.struts.action.ActionMapping', 'org.hibernate.Query', 'java.util.ArrayList', 'org.apache.struts.action.ActionForward', 'java.util.HashSet', 'org.apache.struts.action.ActionForm', 'org.hibernate.Session', 'javax.servlet.http.HttpServletResponse'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.Session'], ['org.hibernate.HibernateException', 'org.hibernate.CacheMode', 'org.hibernate.SessionFactory', 'org.hibernate.JDBCException', 'org.hibernate.Session'], ['org.hibernate.SessionFactory', 'hibernate.util.HibernateUtil', 'org.hibernate.Session', 'org.hibernate.Transaction'], ['net.sf.gilead.core.hibernate.HibernateUtil', 'net.sf.gilead.core.PersistentBeanManager', 'com.google.gwt.user.server.rpc.RemoteServiceServlet', 'org.hibernate.SessionFactory', 'hibernate.GreetingService', 'org.hibernate.Session', 'net.sf.gilead.core.store.stateless.StatelessProxyStore', 'org.hibernate.Transaction'], ['org.hibernate.annotations.NaturalId', 'javax.persistence.Version', 'java.io.Serializable', 'javax.persistence.Entity'], ['java.util.List', 'org.hibernate.Session', 'org.hibernate.SessionFactory']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# JodaTime Zero-shot Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot/jodatime"
code_name = "jodatime"
codes = jodatime_codes
predicted_imports = [['org.joda.time.format.DateTimeFormatter', 'org.joda.time.DateTimeZone', 'org.joda.time.DateTime', 'java.util.TimeZone', 'org.joda.time.format.ISODateTimeFormat'], ['org.joda.time.DateMidnight', 'org.joda.time.DateTimeZone', 'org.joda.time.DateTime'], ['org.joda.time.Interval', 'org.joda.time.PeriodFormatterBuilder', 'org.joda.time.PeriodFormatter'], ['org.joda.time.Period', 'org.joda.time.ReadableInstant'], ['org.joda.time.DateTimeZone', 'org.joda.time.DateTime', 'org.joda.time.Chronology', 'org.joda.time.chrono.GJChronology'], ['org.joda.time.PeriodType', 'org.joda.time.Duration', 'org.joda.time.PeriodBuilder', 'org.joda.time.PeriodFormatterBuilder'], ['org.joda.time.DateTime', 'java.text.DateFormat', 'java.util.Calendar', 'org.joda.time.Days', 'java.text.ParseException', 'java.util.Date', 'java.text.SimpleDateFormat'], ['java.util.concurrent.TimeUnit', 'org.joda.time.Period', 'org.joda.time.PeriodType'], ['org.joda.time.DateTimeZone', 'java.util.TimeZone'], ['java.io.ObjectInputStream', 'java.io.Serializable', 'java.io.IOException', 'org.joda.time.DateTimeZone', 'java.io.ObjectOutputStream', 'org.joda.time.chrono.ISOChronology'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'java.text.SimpleDateFormat', 'java.text.ParseException'], ['java.text.SimpleDateFormat', 'java.util.Date', 'org.joda.time.DateTime', 'java.text.ParseException'], ['org.junit.Assert.assertEquals', 'org.joda.time.format.PeriodFormatterBuilder', 'org.junit.Test', 'org.joda.time.format.PeriodFormatter', 'org.joda.time.Period'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime'], ['org.joda.time.DateTime', 'org.joda.time.MutableDateTime'], ['org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.DateTime', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatter'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormatterBuilder', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.DateTimeZone'], ['org.joda.time.Interval', 'org.joda.time.Instant', 'org.joda.time.ReadableInstant', 'java.util.Date'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTimeZone', 'org.joda.time.DateTime'], ['org.joda.time.DateTimeZone', 'org.joda.time.DateTime', 'java.text.DateFormat', 'org.joda.time.LocalDateTime', 'java.text.ParseException', 'java.util.Date', 'java.text.SimpleDateFormat'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime', 'java.text.DateFormat', 'java.text.ParseException', 'java.util.Date', 'java.text.SimpleDateFormat'], ['org.joda.time.LocalDate', 'org.joda.time.DateTimeConstants'], ['org.joda.time.format.PeriodFormat', 'org.joda.time.DateTime', 'org.joda.time.Period', 'org.joda.time.PeriodType'], ['org.joda.time.DateMidnight', 'org.joda.time.Days', 'org.joda.time.LocalDateTime'], ['org.joda.time.Duration', 'org.joda.time.Period'], ['java.util.ArrayList', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.DateTime', 'java.util.List', 'org.joda.time.Days', 'java.util.ArrayList', 'org.joda.time.LocalDate'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.DateTime'], ['org.joda.time.DateTimeZone', 'org.joda.time.DateTime'], ['org.joda.time.DateTime', 'org.joda.time.Weeks'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.DateTime'], ['org.joda.time.Period', 'org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.format.PeriodFormatter'], ['org.joda.time.format.PeriodFormatter', 'org.joda.time.Period', 'java.util.Locale', 'org.joda.time.format.PeriodFormat'], ['org.joda.time.DateTime', 'org.joda.time.Period', 'org.joda.time.ReadablePeriod', 'org.joda.time.PeriodFormatterBuilder', 'org.joda.time.PeriodFormatter'], ['org.joda.time.format.ISOPeriodFormat', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.format.PeriodFormatter'], ['java.util.concurrent.TimeUnit', 'org.joda.time.DateTimeZone', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime'], ['org.joda.time.DateTime'], ['org.joda.time.Days', 'org.joda.time.DateTime', 'java.util.Date'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime'], ['org.joda.time.LocalDate'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime', 'org.joda.time.format.DateTimeParser', 'org.joda.time.format.DateTimeFormatterBuilder'], ['org.joda.time.Period', 'org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.format.PeriodFormatter'], ['java.util.Set', 'org.joda.time.DateTimeZone', 'org.joda.time.tz.Provider', 'java.util.TimeZone', 'java.util.HashSet', 'java.util.Date'], ['org.joda.time.DateTimeZone', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime'], ['org.joda.time.DateTime', 'org.joda.time.MillisProvider'], ['org.joda.time.format.DateTimeFormat', 'org.joda.time.LocalDate', 'org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.DateTime']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# GWT Zero-shot Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot/gwt"
code_name = "gwt"
codes = gwt_codes
predicted_imports = [['com.google.gwt.user.client.ui.Composite', 'com.google.gwt.event.dom.client.MouseDownEvent', 'com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.event.dom.client.MouseDownHandler'], ['com.google.gwt.junit.client.GWTTestCase', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.sample.stockwatcher.client.GreetingService', 'com.google.gwt.sample.stockwatcher.client.GreetingServiceAsync', 'com.google.gwt.core.client.GWT'], ['com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.Window'], ['com.google.gwt.user.client.ui.VerticalSplitPanel', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.HTML'], ['com.google.gwt.core.client.JsArrayInteger', 'com.google.gwt.core.client.JsArray'], ['com.google.gwt.user.client.DOM', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.Element'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Table', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.Panel', 'com.google.gwt.user.client.Timer'], ['com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.user.client.Window'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.Command', 'com.google.gwt.user.client.ui.MenuBar'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.ClickListener', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.FormPanel', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.core.client.EntryPoint'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.ui.ClickListener', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.core.client.EntryPoint'], ['com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.core.client.GWT', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.user.client.Window'], ['com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.URL', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestException'], ['com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.DockPanel', 'com.google.gwt.user.client.ui.AbstractImagePrototype', 'com.google.gwt.user.client.ui.ImageBundle', 'com.google.gwt.user.client.ui.Image'], ['com.google.gwt.core.client.GWT', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.uibinder.client.UiBinder', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.Window'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.PopupPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.uibinder.client.UiBinder', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.core.client.GWT', 'com.google.gwt.uibinder.client.UiField', 'com.google.gwt.uibinder.client.UiHandler', 'com.google.gwt.user.client.ui.Label'], ['java.util.UUID', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestException'], ['com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.DOM', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.user.client.ui.HasClickHandlers'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.KeyUpEvent', 'com.google.gwt.event.dom.client.KeyUpHandler', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.DialogBox', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.event.dom.client.KeyCodes', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.event.dom.client.FormSubmitEvent', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.event.dom.client.FormHandler', 'com.google.gwt.event.dom.client.FormSubmitCompleteEvent', 'com.google.gwt.event.dom.client.ClickListener', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.FormPanel'], ['com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.dom.client.Document', 'com.google.gwt.dom.client.NodeList', 'com.google.gwt.core.client.GWT'], ['com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Response', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.user.client.Window'], ['com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.http.client.JsonpRequestBuilder', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.Window'], ['com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.DialogBox', 'com.google.gwt.user.client.ui.Grid', 'com.google.gwt.user.client.ui.HTML'], ['com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.URL', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestException'], ['com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.URL', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.user.client.Window'], ['com.google.gwt.user.client.ui.VerticalPanel', 'java.util.Iterator', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.Grid', 'java.util.List', 'com.google.gwt.user.client.ui.CheckBox', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.HTML', 'java.util.ArrayList', 'com.google.gwt.user.client.ui.DockPanel', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.RadioButton', 'com.google.gwt.user.client.ui.HorizontalPanel'], ['com.google.gwt.dom.client.DOM', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.FormPanel', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.SubmitHandler', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.event.dom.client.SubmitEvent'], ['com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.user.client.ui.FlowPanel', 'com.google.gwt.animation.client.Animation', 'com.google.gwt.dom.client.Element', 'com.google.gwt.core.client.GWT'], ['com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.user.client.ui.SimplePanel'], ['com.google.gwt.http.client.Request', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.URL', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.core.client.JsArray', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.http.client.Response', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.user.client.Window'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.Hyperlink', 'com.google.gwt.event.dom.client.HasClickHandlers'], ['com.google.gwt.user.server.rpc.RemoteServiceServlet', 'javax.servlet.ServletOutputStream', 'java.io.FileInputStream', 'java.io.File', 'javax.servlet.ServletContext', 'javax.servlet.http.HttpServletResponse', 'java.io.DataInputStream'], ['com.google.gwt.user.client.ui.HorizontalPanel', 'com.sencha.gxt.widget.core.client.container.LayoutContainer', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.Element'], ['com.google.gwt.http.client.Request', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.URL', 'com.google.gwt.http.client.Response', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.user.client.Window'], ['com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.event.dom.client.MouseOutHandler', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.MouseOutEvent', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.core.client.RunAsyncCallback', 'com.google.gwt.user.client.ui.DockLayoutPanel', 'com.google.gwt.core.client.GWT', 'com.google.gwt.user.client.ui.ResizeComposite', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.Window'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.DialogBox', 'com.google.gwt.user.client.ui.HTML'], ['com.google.gwt.user.client.ui.Widget', 'com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.LayoutPanel', 'com.google.gwt.user.client.ui.RootLayoutPanel', 'com.google.gwt.core.client.EntryPoint'], ['com.google.gwt.event.logical.shared.BeforeSelectionHandler', 'com.google.gwt.user.client.ui.TabBar', 'com.google.gwt.event.logical.shared.SelectionEvent', 'com.google.gwt.event.logical.shared.SelectionHandler', 'com.google.gwt.event.logical.shared.BeforeSelectionEvent', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.Window'], ['com.google.gwt.dom.client.Document', 'com.google.gwt.dom.client.Element', 'java.util.ArrayList', 'com.google.gwt.user.client.DOM', 'com.google.gwt.dom.client.BodyElement'], ['com.google.gwt.event.dom.client.BlurHandler', 'com.google.gwt.event.dom.client.KeyDownEvent', 'com.google.gwt.user.client.ui.HasHTML', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.BlurEvent', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.HasText', 'com.google.gwt.user.client.ui.HasKeyDownHandlers', 'com.google.gwt.user.client.DOM', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.KeyDownHandler', 'com.google.gwt.user.client.ui.ComplexPanel', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.user.client.ui.HasClickHandlers']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# XStream Zero-shot Self-consistency

save_directory = "/home/azmain/code_for_compilation_test/zero-shot/xstream"
code_name = "xstream"
codes = xstream_codes
predicted_imports = [['com.thoughtworks.xstream.converters.UnmarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.MarshallingContext'], ['java.io.IOException', 'com.thoughtworks.xstream.XStream', 'java.io.FileReader', 'java.io.BufferedReader', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['java.util.ArrayList', 'java.util.List', 'com.thoughtworks.xstream.io.json.JettisonMappedXmlDriver', 'com.thoughtworks.xstream.XStream'], ['java.util.ArrayList', 'com.thoughtworks.xstream.XStream'], ['java.io.FileNotFoundException', 'com.thoughtworks.xstream.io.xml.DomDriver', 'java.io.PrintWriter', 'com.thoughtworks.xstream.XStream'], ['java.io.OutputStream', 'com.thoughtworks.xstream.XStream', 'java.io.File', 'java.io.FileOutputStream', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['com.thoughtworks.xstream.annotations.XStreamAsAttribute'], ['android.app.Activity', 'android.os.Bundle', 'org.ksoap2.transport.HttpTransportSE', 'com.thoughtworks.xstream.XStream', 'org.ksoap2.SoapEnvelope', 'org.ksoap2.serialization.SoapObject', 'org.ksoap2.serialization.SoapSerializationEnvelope', 'com.thoughtworks.xstream.io.xml.DomDriver', 'android.widget.TextView'], ['java.io.FileNotFoundException', 'java.io.FileInputStream', 'com.thoughtworks.xstream.io.xml.DomDriver', 'com.thoughtworks.xstream.XStream'], ['java.io.StringWriter', 'com.thoughtworks.xstream.io.xml.PrettyPrintWriter', 'com.thoughtworks.xstream.XStream'], ['java.util.List', 'com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'com.thoughtworks.xstream.converters.collections.CollectionConverter', 'com.thoughtworks.xstream.mapper.ClassAliasingMapper'], ['java.util.List', 'com.thoughtworks.xstream.MarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.UnmarshallingContext', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter'], ['com.thoughtworks.xstream.mapper.ClassAliasingMapper', 'com.thoughtworks.xstream.mapper.Mapper'], ['com.thoughtworks.xstream.converters.SingleValueConverter', 'com.thoughtworks.xstream.converters.enums.AbstractSingleValueConverter'], ['com.thoughtworks.xstream.io.xml.MarshallingContext', 'com.thoughtworks.xstream.XStream', 'java.util.HashSet', 'com.thoughtworks.xstream.converters.collections.CollectionConverter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter'], ['com.thoughtworks.xstream.mapper.MapperWrapper', 'com.thoughtworks.xstream.XStream'], [], ['com.thoughtworks.xstream.MarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'java.util.HashMap', 'com.thoughtworks.xstream.UnmarshallingContext', 'com.thoughtworks.xstream.converters.Converter', 'java.util.Map', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter'], ['com.thoughtworks.xstream.MarshallingContext', 'com.thoughtworks.xstream.UnmarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter'], ['java.nio.charset.Charset', 'java.io.InputStreamReader', 'java.io.InputStream', 'com.thoughtworks.xstream.XStream', 'java.io.FileInputStream', 'java.io.File', 'java.io.Reader'], ['com.thoughtworks.xstream.converters.reflection.ReflectionConverter', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.converters.UnmarshallingContext', 'com.thoughtworks.xstream.converters.reflection.ReflectionProvider', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.Converter', 'java.util.Map', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.MarshallingContext'], ['java.util.Arrays', 'com.thoughtworks.xstream.annotations.XStreamImplicit', 'java.util.List', 'com.thoughtworks.xstream.XStream', 'java.io.File', 'com.thoughtworks.xstream.annotations.XStreamConverter', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.converters.extended.ToAttributedValueConverter'], ['com.thoughtworks.xstream.io.xml.XppDriver', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.converters.javabean.JavaBeanConverter', 'com.thoughtworks.xstream.XStream'], ['com.thoughtworks.xstream.converters.reflection.ReflectionConverter', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.converters.UnmarshallingContext', 'com.thoughtworks.xstream.converters.reflection.ReflectionProvider', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.MarshallingContext'], ['com.thoughtworks.xstream.converters.extended.ToAttributedValueConverter', 'com.thoughtworks.xstream.annotations.XStreamConverter', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.XStream'], ['java.util.ArrayList', 'java.util.List', 'java.util.Collection', 'com.thoughtworks.xstream.converters.SingleValueConverter'], ['xstream.Content', 'com.thoughtworks.xstream.io.xml.MarshallingContext', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.io.xml.UnmarshallingContext', 'org.apache.commons.lang3.text.WordUtils'], ['com.thoughtworks.xstream.MarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'java.util.HashMap', 'com.thoughtworks.xstream.UnmarshallingContext', 'com.thoughtworks.xstream.converters.Converter', 'java.util.Map', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter'], ['java.util.ArrayList', 'java.util.List', 'com.thoughtworks.xstream.io.xml.DomDriver', 'com.thoughtworks.xstream.XStream'], ['com.thoughtworks.xstream.io.UnmarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.MarshallingContext', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter'], ['com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.XStream'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.hibernate.converter.HibernatePersistentSortedMapConverter', 'com.thoughtworks.xstream.hibernate.converter.HibernateProxyConverter', 'com.thoughtworks.xstream.hibernate.converter.HibernatePersistentSortedSetConverter', 'com.thoughtworks.xstream.hibernate.converter.HibernatePersistentMapConverter', 'com.thoughtworks.xstream.hibernate.converter.HibernateMapper', 'com.thoughtworks.xstream.hibernate.converter.HibernatePersistentCollectionConverter', 'com.thoughtworks.xstream.mapper.MapperWrapper'], ['com.thoughtworks.xstream.io.xml.DomDriver', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.XStream'], ['java.util.Iterator', 'com.thoughtworks.xstream.annotations.XStreamImplicit', 'java.util.List', 'com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'com.thoughtworks.xstream.annotations.XStreamAlias'], ['com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.converters.ConverterLookup', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.converters.reflection.ReflectionProvider', 'com.thoughtworks.xstream.converters.SingleValueConverter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.mapper.MapperWrapper'], ['com.thoughtworks.xstream.io.xml.StaxDriver', 'java.util.Map', 'java.util.HashMap', 'com.thoughtworks.xstream.XStream'], ['com.thoughtworks.xstream.io.UnmarshallingContext', 'java.util.ArrayList', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.io.MarshallingContext', 'com.thoughtworks.xstream.convert.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter'], ['java.util.ArrayList', 'java.util.List', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.XStream'], ['com.thoughtworks.xstream.io.xml.XppDriver', 'com.thoughtworks.xstream.io.xml.PrettyPrintWriter', 'com.thoughtworks.xstream.XStream', 'java.io.Writer', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.io.QuickWriter'], ['com.thoughtworks.xstream.MarshallingContext', 'com.thoughtworks.xstream.UnmarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter'], ['java.util.List', 'com.thoughtworks.xstream.io.xml.StaxDriver', 'com.thoughtworks.xstream.XStream'], ['java.util.List', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.annotations.XStreamImplicit'], ['java.util.ArrayList', 'java.util.List', 'com.thoughtworks.xstream.io.xml.DomDriver', 'com.thoughtworks.xstream.XStream'], ['java.io.FileOutputStream', 'java.io.File', 'com.thoughtworks.xstream.XStream']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

#### Self-consistency from 20 Sample

In [ ]:
# Android Zero-shot Self-consistency from 20 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-20-sample"
code_name = "android"
codes = android_codes
predicted_imports = [['android.widget.TextView', 'android.os.Bundle', 'android.app.Activity'], ['android.os.Bundle', 'android.app.Activity'], ['com.google.android.maps.OverlayItem', 'android.graphics.drawable.Drawable', 'com.google.android.maps.GeoPoint', 'com.google.android.maps.ItemizedOverlay'], ['android.widget.TextView', 'android.os.Bundle', 'android.app.Activity'], ['android.widget.TabSpec', 'android.graphics.drawable.Drawable', 'android.widget.TabHost'], ['android.widget.LinearLayout', 'android.view.ViewGroup', 'android.view.Gravity'], ['android.os.Bundle', 'android.view.View', 'android.app.Activity', 'android.widget.ArrayAdapter'], ['java.io.IOException', 'java.net.Inet4Address', 'java.net.InetAddress', 'java.net.UnknownHostException', 'java.net.Socket', 'android.util.Log'], ['android.widget.TextView', 'android.os.Bundle', 'java.util.Timer', 'java.util.TimerTask'], ['android.widget.Chronometer', 'android.util.Log', 'android.os.SystemClock'], ['com.google.android.maps.MyLocationOverlay', 'com.google.android.maps.GeoPoint', 'com.google.android.maps.MapView', 'android.content.Context', 'android.location.Location', 'android.graphics.Canvas'], ['android.os.Bundle', 'android.app.Activity', 'android.graphics.Color', 'android.widget.LinearLayout', 'android.widget.TextView', 'android.view.ViewGroup'], ['java.io.IOException', 'android.graphics.BitmapFactory', 'java.io.ByteArrayOutputStream', 'android.net.Uri', 'android.graphics.Bitmap', 'java.io.FileNotFoundException', 'android.content.ContentResolver', 'java.io.ByteArrayInputStream'], ['java.io.IOException', 'java.net.HttpURLConnection', 'java.net.URL', 'java.io.InputStream', 'java.io.FileNotFoundException', 'java.io.File', 'java.io.FileOutputStream', 'java.net.ProtocolException', 'java.net.MalformedURLException'], ['android.net.Uri', 'android.content.Intent'], ['android.widget.SeekBar', 'android.view.View', 'android.content.Context', 'android.view.MotionEvent', 'android.graphics.Canvas', 'android.util.AttributeSet', 'android.util.Log'], ['java.io.IOException', 'java.io.InputStreamReader', 'android.util.Log', 'org.apache.http.HttpEntity', 'org.apache.http.client.HttpClient', 'org.apache.http.impl.client.DefaultHttpClient', 'java.net.URI', 'java.net.URISyntaxException', 'org.apache.http.client.methods.HttpGet', 'org.apache.http.HttpResponse'], ['android.content.Intent', 'android.os.Bundle', 'android.app.Activity', 'android.view.View.OnClickListener', 'android.content.DialogInterface'], ['android.widget.GridView', 'android.view.View', 'android.content.Context', 'android.view.ViewGroup', 'android.widget.ImageView'], ['android.widget.TextView', 'android.view.View', 'android.content.Context', 'android.view.animation.ScaleAnimation'], ['android.widget.ImageButton', 'android.widget.LinearLayout', 'android.content.Context'], ['android.content.Intent', 'android.app.ListActivity', 'android.os.Bundle', 'android.view.View', 'android.provider.Contacts.People', 'android.database.Cursor', 'android.widget.SimpleCursorAdapter', 'android.widget.ListView'], ['java.io.IOException', 'android.graphics.BitmapFactory', 'java.net.HttpURLConnection', 'android.os.Bundle', 'android.app.Activity', 'android.widget.ListView', 'java.net.URL', 'android.graphics.Bitmap', 'java.util.Random', 'java.net.MalformedURLException', 'android.widget.ImageView', 'android.widget.ArrayAdapter', 'java.util.ArrayList'], ['android.location.LocationManager', 'android.os.Bundle', 'android.content.Context', 'android.app.Activity', 'java.util.Iterator', 'java.util.List', 'android.util.Log'], ['android.app.ListActivity', 'android.os.Bundle', 'android.content.Context', 'android.database.sqlite.SQLiteDatabase', 'java.util.ArrayList'], ['android.view.View', 'android.view.KeyEvent', 'android.content.Context', 'android.widget.ScrollView', 'android.view.MotionEvent', 'android.view.View.OnTouchListener'], ['android.widget.GridView', 'android.view.View', 'android.content.Context', 'android.widget.BaseAdapter', 'android.widget.TextView', 'android.view.ViewGroup'], ['android.app.Application', 'android.content.Context'], ['android.os.PowerManager', 'android.os.IBinder', 'android.content.Intent', 'android.content.Context', 'android.app.Service'], ['org.apache.http.util.EntityUtils', 'java.io.IOException', 'org.apache.http.HttpResponse', 'org.apache.http.HttpEntity', 'org.apache.http.client.HttpClient', 'org.apache.http.impl.client.DefaultHttpClient', 'org.apache.http.HttpHost', 'org.apache.http.client.methods.HttpGet'], ['android.os.PowerManager', 'android.content.Context'], ['java.io.IOException', 'java.io.File', 'android.os.Environment', 'android.media.MediaRecorder'], ['java.io.IOException', 'android.view.SurfaceHolder', 'android.graphics.PixelFormat', 'android.content.Context', 'android.hardware.Camera', 'android.hardware.Camera.Parameters'], ['android.app.TabActivity', 'android.os.Bundle', 'android.view.View', 'android.widget.TabHost', 'android.widget.TabHost.OnTabChangeListener', 'android.view.View.OnClickListener', 'android.widget.TabHost.TabContentFactory', 'android.widget.ListView', 'android.util.Log'], ['java.io.IOException', 'java.io.File', 'android.media.MediaRecorder'], ['android.content.ContentValues', 'android.database.sqlite.SQLiteDatabase'], ['android.os.Message', 'android.os.Handler', 'android.app.Activity', 'android.app.ProgressDialog'], ['java.io.IOException', 'java.io.FileInputStream', 'java.net.HttpURLConnection', 'java.io.DataInputStream', 'java.net.URL', 'java.io.DataOutputStream', 'java.io.InputStream', 'java.io.File', 'java.net.MalformedURLException'], ['com.google.android.maps.MyLocationOverlay', 'com.google.android.maps.Overlay', 'com.google.android.maps.MapActivity', 'android.os.Bundle', 'com.google.android.maps.MapView', 'android.app.Activity', 'com.google.android.maps.MapController', 'java.util.List'], ['android.util.Log', 'android.os.Bundle', 'android.view.View', 'android.app.Activity', 'android.view.View.OnClickListener', 'android.widget.ImageView', 'android.widget.SlidingDrawer'], ['android.widget.TextView', 'android.view.View', 'android.content.Context', 'android.graphics.Color'], ['java.io.IOException', 'android.view.SurfaceHolder', 'android.os.Bundle', 'android.view.View', 'android.content.pm.ActivityInfo', 'android.webkit.URLUtil', 'android.view.SurfaceView', 'java.net.URL', 'android.widget.Button', 'android.content.Intent', 'java.io.InputStream', 'java.io.File', 'java.io.FileOutputStream', 'java.net.URLConnection', 'android.graphics.PixelFormat', 'android.app.Activity', 'android.net.Uri', 'android.media.MediaPlayer', 'android.util.Log'], ['org.apache.http.util.EntityUtils', 'java.io.IOException', 'org.apache.http.HttpResponse', 'org.apache.http.HttpEntity', 'org.apache.http.client.HttpClient', 'org.apache.http.impl.client.DefaultHttpClient', 'org.apache.http.HttpHost', 'org.apache.http.client.methods.HttpGet'], ['android.app.Service', 'android.os.IBinder', 'android.content.Intent'], ['android.content.BroadcastReceiver', 'android.content.Intent', 'android.content.ComponentName', 'android.os.Bundle', 'android.content.Context', 'android.app.Activity', 'android.webkit.MalformedMimeTypeException', 'android.content.IntentFilter', 'android.util.Log'], ['android.widget.Button', 'android.content.Intent', 'android.widget.EditText', 'android.os.Bundle', 'android.view.View', 'android.app.Activity'], ['org.apache.http.client.ClientProtocolException', 'java.io.IOException', 'android.os.Bundle', 'android.app.Activity', 'org.apache.http.impl.client.DefaultHttpClient', 'android.widget.TextView', 'org.apache.http.client.methods.HttpGet'], ['android.graphics.BitmapFactory', 'android.app.AlertDialog', 'android.content.Context', 'android.hardware.Camera', 'android.graphics.Bitmap', 'android.app.Dialog', 'android.widget.TextView', 'android.widget.ImageView', 'android.hardware.Camera.PictureCallback'], ['androidExamples.R', 'android.content.Context', 'android.widget.LinearLayout', 'android.view.Gravity', 'android.view.LayoutInflater', 'android.util.AttributeSet'], ['android.widget.Button', 'android.widget.EditText', 'android.os.Bundle', 'android.view.View', 'android.app.Activity', 'android.widget.TextView', 'java.text.NumberFormat', 'android.util.Log']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# JDK Zero-shot Self-consistency from 20 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-20-sample"
code_name = "jdk"
codes = jdk_codes
predicted_imports = [['java.awt.event', 'java.awt', 'javax.swing', 'java.awt.image', 'java.io'], ['java.awt.BorderLayout', 'javax.swing.BorderFactory', 'javax.swing.JSplitPane', 'java.awt.image.BufferedImage', 'java.awt.event.ActionEvent', 'java.awt.GridBagLayout', 'java.awt.event.ActionListener', 'java.awt.FlowLayout', 'java.awt.Color', 'javax.swing.table.DefaultTableModel', 'java.awt.Graphics2D', 'javax.swing.JPanel', 'javax.swing.SwingUtilities', 'javax.swing.JScrollPane', 'java.awt.GridLayout', 'javax.swing.ImageIcon', 'javax.swing.JCheckBox', 'javax.swing.JComboBox', 'javax.swing.JFrame', 'javax.swing.JLabel', 'java.awt.Dimension', 'javax.swing.border.TitledBorder', 'javax.swing.UIManager', 'java.awt.GradientPaint', 'javax.swing.JButton', 'javax.swing.JTable'], ['java.math.RoundingMode', 'java.text.DecimalFormat', 'java.text.ParseException', 'java.util.Locale', 'java.text.DecimalFormatSymbols'], ['java.util.ArrayList', 'java.util.regex.Pattern', 'java.util.List', 'java.util.regex.Matcher'], ['java.util.HashMap', 'java.util.TreeMap', 'java.util.Map', 'java.util.LinkedList', 'java.util.List', 'java.util.Collections', 'java.util.Comparator', 'java.util.LinkedHashMap'], ['java.util.Observable', 'java.awt.Graphics', 'java.awt.BorderLayout', 'java.awt.Color', 'java.util.Observer', 'java.awt.RenderingHints', 'java.awt.Graphics2D', 'javax.swing.JFrame', 'javax.swing.JLabel', 'javax.swing.JPanel', 'java.util.Random', 'java.awt.event.ActionEvent', 'java.awt.EventQueue', 'javax.swing.JButton', 'javax.swing.Icon', 'java.awt.event.ActionListener', 'java.awt.Component'], ['java.text.SimpleDateFormat', 'java.util.ArrayList', 'java.text.ParseException', 'java.util.Date', 'java.util.Collections', 'java.util.Comparator'], [], ['java.sql.Connection', 'java.sql.DriverManager', 'java.sql.SQLException', 'java.util.Properties'], ['java.util.Locale', 'java.text.NumberFormat'], ['java.io.BufferedWriter', 'javax.swing.JFileChooser', 'java.io.File', 'java.io.FileWriter'], ['java.text.SimpleDateFormat', 'java.text.DateFormat', 'java.text.ParseException', 'java.util.GregorianCalendar', 'java.util.Date', 'java.util.Calendar'], ['javax.swing.JFrame', 'javax.swing.SwingUtilities', 'javax.swing.JTextArea', 'javax.swing.UIManager'], ['java.lang.reflect.Method'], ['java.util.HashMap'], ['java.lang.reflect.Method', 'java.util.HashMap', 'java.util.ArrayList', 'java.util.prefs.Preferences', 'java.util.Map', 'java.lang.reflect.InvocationTargetException', 'java.util.List'], ['java.text.SimpleDateFormat', 'java.time.temporal.TemporalAccessor', 'java.text.ParseException', 'java.util.Date', 'java.time.format.DateTimeFormatter'], ['java.lang.reflect.Field', 'java.lang.reflect.Array'], ['java.util.concurrent.ThreadLocalRandom', 'java.util.Iterator', 'java.util.Random', 'java.util.Collection', 'java.util.ArrayList', 'java.util.NoSuchElementException', 'java.util.List'], ['java.io.FileInputStream', 'java.io.File', 'java.io.IOException', 'java.util.ArrayList', 'java.io.BufferedInputStream', 'java.io.FileOutputStream', 'java.io.DataInputStream', 'java.io.BufferedOutputStream', 'java.io.DataOutputStream'], ['java.awt.event', 'javax.imageio', 'javax.swing.border', 'java.awt', 'javax.swing', 'java.awt.image', 'java.io'], ['java.text.DecimalFormat', 'java.awt.Container', 'javax.swing.JFrame', 'javax.swing.JLabel', 'java.beans.PropertyChangeEvent', 'javax.swing.SwingWorker', 'java.awt.EventQueue', 'java.util.List', 'java.awt.GridLayout', 'java.beans.PropertyChangeListener', 'javax.swing.JProgressBar'], []]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# Hibernate Zero-shot Self-consistency from 20 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-20-sample"
code_name = "hibernate"
codes = hibernate_codes
predicted_imports = [['org.hibernate.SessionFactory', 'org.hibernate.cfg.AnnotationConfiguration'], ['hibernate.UserDAO', 'org.hibernate.SessionFactory', 'hibernate.User', 'org.hibernate.Transaction', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration'], ['javax.persistence.OneToMany', 'javax.persistence.CacheConcurrencyStrategy', 'java.util.List', 'javax.persistence.ManyToOne', 'javax.persistence.Column', 'javax.persistence.Id', 'javax.persistence.Cache', 'javax.persistence.JoinColumn', 'java.io.Serializable', 'javax.persistence.Entity'], ['org.hibernate.type.Hibernate', 'org.hibernate.dialect.function.StandardSQLFunction', 'java.sql.Types', 'org.hibernate.dialect.Dialect', 'org.hibernate.dialect.function.VarArgsSQLFunction', 'org.hibernate.dialect.function.SQLFunctionTemplate'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'org.hibernate.HibernateException'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory'], ['org.hibernate.cfg.Configuration', 'java.util.List', 'org.hibernate.SessionFactory', 'org.dom4j.DocumentException', 'java.net.MalformedURLException', 'java.util.Iterator', 'org.hibernate.engine.spi.EntityMode', 'org.dom4j.Document', 'org.hibernate.Transaction', 'org.hibernate.HibernateException', 'java.io.File', 'org.dom4j.io.SAXReader', 'org.hibernate.cfg.SchemaExport', 'org.hibernate.Session'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration'], ['javax.persistence.Temporal', 'javax.persistence.Column', 'java.util.Date', 'javax.validation.constraints.AssertTrue', 'javax.persistence.TemporalType', 'javax.persistence.MappedSuperclass', 'javax.validation.constraints.NotNull', 'javax.persistence.Transient'], ['java.util.List', 'java.io.Serializable', 'org.hibernate.Session'], ['java.util.List', 'java.io.Serializable', 'org.hibernate.Session'], ['javax.persistence.ManyToOne', 'javax.persistence.Column', 'javax.persistence.Id', 'javax.persistence.JoinColumn', 'java.io.Serializable', 'javax.persistence.FetchType', 'javax.persistence.ForeignKey', 'javax.persistence.Entity', 'javax.persistence.Table'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'javax.faces.context.FacesContext', 'org.hibernate.Transaction', 'org.hibernate.Session'], ['org.hibernate.annotations.IndexColumn', 'javax.persistence.OneToMany', 'java.util.List', 'javax.persistence.ManyToOne', 'javax.persistence.Column', 'javax.persistence.ManyToMany', 'javax.persistence.Id', 'java.util.Collection', 'javax.persistence.CascadeType', 'javax.persistence.JoinColumn', 'javax.persistence.JoinTable', 'java.io.Serializable', 'javax.persistence.FetchType', 'javax.persistence.ForeignKey', 'javax.persistence.Entity', 'javax.persistence.Table'], ['javax.persistence.ManyToOne', 'javax.persistence.Column', 'javax.persistence.ManyToMany', 'javax.persistence.Id', 'java.util.Collection', 'javax.persistence.CascadeType', 'javax.persistence.JoinColumn', 'java.io.Serializable', 'javax.persistence.FetchType', 'javax.persistence.ForeignKey', 'javax.persistence.Entity', 'javax.persistence.Table'], ['java.io.InputStream', 'java.util.Date', 'java.sql.Blob', 'java.io.ByteArrayOutputStream', 'org.hibernate.Hibernate', 'java.sql.SQLException', 'java.io.IOException'], ['hibernate.Event', 'java.util.List', 'org.hibernate.Session'], [], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory'], ['javax.persistence.Column', 'javax.persistence.Id', 'javax.persistence.GeneratedValue', 'javax.persistence.Index', 'javax.persistence.Entity', 'org.hibernate.annotations.Table'], ['org.hibernate.SessionFactory', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.Transaction', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'com.google.gwt.json.client.JSONObject', 'org.hibernate.query.Query', 'org.hibernate.Session'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'org.hibernate.HibernateException', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'java.lang.reflect.Method', 'java.lang.reflect.InvocationHandler'], ['org.hibernate.annotations.FetchMode', 'javax.persistence.Id', 'javax.persistence.CascadeType', 'javax.persistence.GeneratedValue', 'javax.persistence.OneToOne', 'javax.persistence.Entity', 'org.hibernate.annotations.Fetch'], ['org.hibernate.SessionFactory', 'java.util.List', 'org.hibernate.tool.hbm2ddl.SchemaExport', 'org.hibernate.annotations.FetchMode', 'javax.persistence.Id', 'javax.persistence.CascadeType', 'org.hibernate.Query', 'javax.persistence.GeneratedValue', 'javax.persistence.OneToOne', 'javax.persistence.Entity', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.annotations.Fetch'], ['org.hibernate.Query', 'java.util.List', 'org.hibernate.Session', 'hibernate.HibernateUtil'], ['javax.persistence.OneToMany', 'java.util.List', 'javax.persistence.IndexColumn', 'javax.persistence.ManyToOne', 'javax.persistence.Column', 'javax.persistence.Id', 'javax.persistence.GeneratedValue', 'javax.persistence.JoinColumn', 'java.io.Serializable', 'java.util.ArrayList', 'javax.persistence.GenerationType', 'javax.persistence.Entity'], ['org.hibernate.SessionFactory', 'java.util.List', 'org.hibernate.criterion.Projections', 'org.hibernate.Criteria', 'org.hibernate.Session'], ['javax.persistence.Basic', 'javax.persistence.Column', 'javax.persistence.Id', 'javax.persistence.NamedQuery', 'java.io.Serializable', 'javax.persistence.Entity', 'javax.persistence.Table', 'javax.persistence.NamedQueries'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'java.util.List', 'org.hibernate.query.Query', 'org.hibernate.Session'], ['javax.persistence.Id', 'javax.persistence.GeneratedValue', 'javax.persistence.Index', 'javax.persistence.Entity', 'javax.persistence.Table'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'org.hibernate.Session'], ['org.hibernate.Transaction', 'org.hibernate.Session'], ['java.util.List', 'java.util.Iterator', 'org.hibernate.Transaction', 'org.hibernate.Query', 'org.hibernate.HibernateException', 'org.apache.log4j.BasicConfigurator', 'org.hibernate.Session'], ['org.hibernate.Transaction', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session'], ['org.hibernate.SessionFactory', 'org.hibernate.HibernateException', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.Transaction', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session'], ['org.hibernate.SessionFactory', 'java.util.List', 'org.springframework.transaction.annotation.Transactional', 'org.hibernate.Transaction', 'java.io.Serializable', 'java.util.Map', 'org.hibernate.Session'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'java.util.List', 'java.util.Iterator', 'org.hibernate.query.Query', 'org.hibernate.Session'], ['java.util.List', 'org.hibernate.Criteria', 'org.hibernate.Transaction', 'org.hibernate.criterion.Restrictions', 'org.hibernate.Session'], ['org.apache.commons.logging.LogFactory', 'org.hibernate.SessionFactory', 'java.util.List', 'javax.naming.InitialContext', 'org.apache.commons.logging.Log', 'org.hibernate.criterion.Example'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'java.util.Properties', 'org.apache.log4j.Logger', 'org.hibernate.Session'], ['org.hibernate.SessionFactory', 'java.util.Set', 'org.hibernate.Session', 'org.apache.struts.actions.DispatchAction', 'org.apache.struts.action.ActionForward', 'javax.servlet.http.HttpSession', 'org.apache.struts.action.ActionMapping', 'java.util.Iterator', 'org.apache.struts.action.ActionForm', 'org.hibernate.Transaction', 'org.hibernate.Query', 'java.util.ArrayList', 'java.util.HashSet', 'javax.servlet.http.HttpServletResponse', 'javax.servlet.http.HttpServletRequest'], ['org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.SessionFactory', 'org.hibernate.JDBCException', 'org.hibernate.CacheMode', 'org.hibernate.HibernateException', 'org.hibernate.Session'], ['org.hibernate.Transaction', 'org.hibernate.SessionFactory', 'org.hibernate.Session', 'hibernate.HibernateUtil'], ['org.hibernate.SessionFactory', 'com.google.gwt.user.client.rpc.RemoteService', 'org.hibernate.Transaction', 'com.google.gwt.user.server.rpc.RemoteServiceServlet', 'org.hibernate.Session'], ['org.hibernate.annotations.NaturalId', 'javax.persistence.Version', 'javax.persistence.Entity', 'java.io.Serializable'], ['java.util.List', 'org.hibernate.SessionFactory', 'org.hibernate.Session']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# JodaTime Zero-shot Self-consistency from 20 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-20-sample"
code_name = "jodatime"
codes = jodatime_codes
predicted_imports = [['org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'org.joda.time.format.DateTimeFormatter', 'java.util.TimeZone', 'org.joda.time.format.ISODateTimeFormat'], ['org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'org.joda.time.DateMidnight'], ['org.joda.time.format.PeriodFormatter', 'org.joda.time.Interval', 'org.joda.time.format.PeriodFormatterBuilder'], ['org.joda.time.Period', 'org.joda.time.ReadableInstant'], ['org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'org.joda.time.chrono.GJChronology', 'org.joda.time.Chronology'], ['org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.Duration', 'org.joda.time.PeriodType'], ['org.joda.time.DateTime', 'java.util.Calendar', 'java.util.Date', 'java.text.ParseException', 'java.text.DateFormat', 'java.text.SimpleDateFormat', 'org.joda.time.Days'], ['org.joda.time.Period', 'org.joda.time.PeriodType', 'java.util.concurrent.TimeUnit'], ['org.joda.time.DateTimeZone', 'java.util.TimeZone'], ['org.joda.time.DateTimeZone', 'java.io.Serializable', 'java.io.ObjectOutputStream', 'java.io.IOException', 'java.io.ObjectInputStream', 'org.joda.time.chrono.ISOChronology'], ['java.text.ParseException', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter', 'java.text.SimpleDateFormat'], ['java.text.ParseException', 'org.joda.time.DateTime', 'java.text.SimpleDateFormat', 'java.util.Date'], ['org.joda.time.Period', 'org.joda.time.format.PeriodFormatterBuilder', 'org.junit.Test', 'org.joda.time.format.PeriodFormatter', 'org.junit.Assert.assertEquals'], ['org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.DateTime', 'org.joda.time.MutableDateTime'], ['org.joda.time.format.PeriodFormatter', 'org.joda.time.DateTime', 'org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.Period'], ['org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.format.DateTimeFormatterBuilder', 'org.joda.time.DateTimeZone', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.Interval', 'org.joda.time.Instant', 'java.util.Date', 'org.joda.time.ReadableInstant'], ['org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'java.util.Date', 'org.joda.time.LocalDateTime', 'java.text.ParseException', 'java.text.DateFormat', 'java.text.SimpleDateFormat'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter', 'java.util.Date', 'java.text.ParseException', 'org.joda.time.format.DateTimeFormat'], ['org.joda.time.DateTimeConstants', 'org.joda.time.LocalDate'], ['org.joda.time.format.PeriodFormat', 'org.joda.time.DateTime', 'org.joda.time.PeriodType', 'org.joda.time.Period'], ['org.joda.time.Days', 'org.joda.time.DateMidnight', 'org.joda.time.LocalDateTime'], ['org.joda.time.Period', 'org.joda.time.Duration'], ['java.util.ArrayList', 'org.joda.time.DateTime'], ['java.util.ArrayList', 'java.util.List', 'org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.Days', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.LocalDate'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.DateTime', 'org.joda.time.DateTimeZone'], ['org.joda.time.DateTime', 'org.joda.time.Weeks'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.format.PeriodFormatter', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatterBuilder'], ['org.joda.time.format.PeriodFormatter', 'org.joda.time.Period', 'java.util.Locale', 'org.joda.time.format.PeriodFormat'], ['org.joda.time.format.PeriodFormatter', 'org.joda.time.DateTime', 'org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.Period'], ['org.joda.time.format.PeriodFormatter', 'org.joda.time.Period', 'org.joda.time.format.ISOPeriodFormat', 'org.joda.time.format.PeriodFormatterBuilder'], ['org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'java.util.concurrent.TimeUnit'], ['org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.DateTime'], ['org.joda.time.DateTime', 'org.joda.time.Days', 'java.util.Date'], ['org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.LocalDate'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormatterBuilder', 'org.joda.time.format.DateTimeParser', 'org.joda.time.format.DateTimeFormat'], ['org.joda.time.format.PeriodFormatter', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatterBuilder'], ['java.util.Set', 'org.joda.time.DateTimeZone', 'java.util.Date', 'java.util.HashSet', 'org.joda.time.tz.Provider', 'java.util.TimeZone'], ['org.joda.time.DateTime', 'org.joda.time.DateTimeZone'], ['org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.MillisProvider', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.LocalDate'], ['org.joda.time.DateTime']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# GWT Zero-shot Self-consistency from 20 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-20-sample"
code_name = "gwt"
codes = gwt_codes
predicted_imports = [['com.google.gwt.user.client.ui.Composite', 'com.google.gwt.event.dom.client.MouseDownEvent', 'com.google.gwt.event.dom.client.MouseDownHandler', 'com.google.gwt.user.client.ui.AbsolutePanel'], ['com.google.gwt.sample.stockwatcher.client.GreetingServiceAsync', 'com.google.gwt.core.client.GWT', 'com.google.gwt.junit.client.GWTTestCase', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.sample.stockwatcher.client.GreetingService'], ['com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.user.client.Window'], ['com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.VerticalSplitPanel'], ['com.google.gwt.core.client.JsArray', 'com.google.gwt.core.client.JsArrayInteger'], ['com.google.gwt.user.client.Element', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.DOM'], ['com.google.gwt.user.client.ui.Panel', 'com.google.gwt.user.client.Timer', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.RootPanel'], ['com.google.gwt.user.client.Window', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.Command', 'com.google.gwt.user.client.ui.MenuBar'], ['com.google.gwt.user.client.ui.FormPanel', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.event.dom.client.ClickListener', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.core.client.EntryPoint'], ['com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.ClickListener'], ['com.google.gwt.user.client.Window', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.core.client.GWT', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder'], ['com.google.gwt.http.client.URL', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder'], ['com.google.gwt.http.client.RequestCallback', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.http.client.Request', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder'], ['com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.AbstractImagePrototype', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.ImageBundle', 'com.google.gwt.user.client.ui.DockPanel', 'com.google.gwt.user.client.ui.Image'], ['com.google.gwt.uibinder.client.UiBinder', 'com.google.gwt.core.client.GWT', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.user.client.ui.Widget'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.Window', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.PopupPanel', 'com.google.gwt.user.client.ui.Button'], ['com.google.gwt.core.client.GWT', 'com.google.gwt.uibinder.client.UiField', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.uibinder.client.UiBinder', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder', 'java.util.UUID'], ['com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.DOM', 'com.google.gwt.event.dom.client.HasClickHandlers', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.event.dom.client.ClickHandler'], ['com.google.gwt.event.dom.client.KeyUpHandler', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.event.dom.client.KeyUpEvent', 'com.google.gwt.user.client.ui.DialogBox', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.event.dom.client.KeyCodes'], ['com.google.gwt.event.dom.client.FormSubmitEvent', 'com.google.gwt.user.client.ui.FormPanel', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.event.dom.client.FormSubmitCompleteEvent', 'com.google.gwt.event.dom.client.FormHandler', 'com.google.gwt.event.dom.client.ClickListener'], ['com.google.gwt.dom.client.Document', 'com.google.gwt.core.client.GWT', 'com.google.gwt.dom.client.NodeList', 'com.google.gwt.dom.client.ButtonElement', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Button'], ['com.google.gwt.user.client.Window', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder'], ['com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.user.client.Window', 'com.google.gwt.jsonp.client.JsonpRequestBuilder'], ['com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.Grid', 'com.google.gwt.user.client.ui.DialogBox'], ['com.google.gwt.http.client.URL', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder'], ['com.google.gwt.user.client.Window', 'com.google.gwt.http.client.URL', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder'], ['java.util.ArrayList', 'java.util.Iterator', 'com.google.gwt.user.client.ui.Grid', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.ClickListener', 'com.google.gwt.user.client.ui.RadioButton', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.DockPanel', 'java.util.List', 'com.google.gwt.user.client.ui.CheckBox'], ['com.google.gwt.event.dom.client.SubmitEvent', 'com.google.gwt.user.client.ui.FormPanel', 'com.google.gwt.event.dom.client.SubmitHandler', 'com.google.gwt.user.client.DOM', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.event.dom.client.ClickHandler'], ['com.google.gwt.animation.client.Animation', 'com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.core.client.GWT', 'com.google.gwt.dom.client.Element', 'com.google.gwt.user.client.ui.FlowPanel'], ['com.google.gwt.user.client.ui.SimplePanel', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.AbsolutePanel'], ['com.google.gwt.user.client.Window', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Request', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder'], ['com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.Hyperlink', 'com.google.gwt.event.dom.client.HasClickHandlers', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Composite'], ['javax.servlet.ServletOutputStream', 'javax.servlet.http.HttpServletResponse', 'javax.servlet.ServletContext', 'java.io.DataInputStream', 'java.io.FileInputStream', 'java.io.File', 'com.google.gwt.user.server.rpc.RemoteServiceServlet'], ['com.sencha.gxt.widget.core.client.container.LayoutContainer', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.dom.client.Element', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.Window', 'com.google.gwt.http.client.URL', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.http.client.RequestBuilder'], ['com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.Composite', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.MouseOutHandler', 'com.google.gwt.event.dom.client.MouseOutEvent', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.core.client.GWT', 'com.google.gwt.core.client.RunAsyncCallback', 'com.google.gwt.user.client.ui.DockLayoutPanel', 'com.google.gwt.user.client.ui.ResizeComposite', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.Window', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Label'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.DialogBox'], ['com.google.gwt.user.client.ui.RootLayoutPanel', 'com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.LayoutPanel', 'com.google.gwt.core.client.EntryPoint'], ['com.google.gwt.event.logical.shared.SelectionEvent', 'com.google.gwt.event.logical.shared.SelectionHandler', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.event.logical.shared.BeforeSelectionEvent', 'com.google.gwt.user.client.ui.TabBar', 'com.google.gwt.user.client.Window', 'com.google.gwt.event.logical.shared.BeforeSelectionHandler'], ['java.util.ArrayList', 'com.google.gwt.dom.client.Document', 'com.google.gwt.dom.client.BodyElement', 'com.google.gwt.dom.client.DOM', 'com.google.gwt.dom.client.Element'], ['com.google.gwt.event.dom.client.BlurEvent', 'com.google.gwt.user.client.ui.HasBlurHandlers', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.event.dom.client.KeyDownHandler', 'com.google.gwt.user.client.DOM', 'com.google.gwt.user.client.ui.HasText', 'com.google.gwt.user.client.ui.HasClickHandlers', 'com.google.gwt.user.client.ui.HasKeyDownHandlers', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.user.client.ui.HasHTML', 'com.google.gwt.event.dom.client.BlurHandler', 'com.google.gwt.user.client.ui.ComplexPanel', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.KeyDownEvent']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# XStream Zero-shot Self-consistency from 20 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-20-sample"
code_name = "xstream"
codes = xstream_codes
predicted_imports = [['com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver', 'java.io.FileReader', 'java.io.IOException', 'java.io.BufferedReader'], ['com.thoughtworks.xstream.io.json.JettisonMappedXmlDriver', 'com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'java.util.List'], ['com.thoughtworks.xstream.XStream', 'java.util.ArrayList'], ['java.io.FileNotFoundException', 'com.thoughtworks.xstream.XStream', 'java.io.PrintWriter', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['java.io.FileOutputStream', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver', 'java.io.OutputStream', 'java.io.File'], ['com.thoughtworks.xstream.annotations.XStreamAsAttribute'], ['org.ksoap2.transport.HttpTransportSE', 'com.thoughtworks.xstream.XStream', 'org.ksoap2.serialization.SoapObject', 'com.thoughtworks.xstream.io.xml.DomDriver', 'android.widget.TextView', 'org.ksoap2.SoapEnvelope', 'android.app.Activity', 'android.os.Bundle', 'org.ksoap2.serialization.SoapSerializationEnvelope'], ['java.io.FileNotFoundException', 'com.thoughtworks.xstream.XStream', 'java.io.FileInputStream', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['com.thoughtworks.xstream.io.xml.PrettyPrintWriter', 'java.io.StringWriter', 'com.thoughtworks.xstream.XStream'], ['com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'java.util.List', 'com.thoughtworks.xstream.converters.collections.CollectionConverter', 'com.thoughtworks.xstream.mapper.ClassAliasingMapper'], ['java.util.List', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.mapper.ClassAliasingMapper'], ['com.thoughtworks.xstream.converters.SingleValueConverter', 'com.thoughtworks.xstream.converters.enums.AbstractSingleValueConverter'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.collections.CollectionConverter', 'java.util.HashSet', 'com.thoughtworks.xstream.converters.MarshallingContext'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.mapper.MapperWrapper'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.MarshallingContext', 'com.thoughtworks.xstream.io.UnmarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.io.HierarchicalStreamReader'], ['java.util.HashMap', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'java.util.Map', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'java.io.InputStreamReader', 'java.io.Reader', 'java.nio.charset.Charset', 'java.io.FileInputStream', 'java.io.InputStream', 'java.io.File'], ['java.util.Map', 'com.thoughtworks.xstream.converters.reflection.ReflectionProvider', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.reflection.ReflectionConverter', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamConverter', 'java.util.Arrays', 'java.util.List', 'com.thoughtworks.xstream.converters.extended.ToAttributedValueConverter', 'com.thoughtworks.xstream.annotations.XStreamImplicit', 'java.io.File', 'com.thoughtworks.xstream.annotations.XStreamAlias'], ['com.thoughtworks.xstream.io.xml.XppDriver', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.converters.extended.JavaBeanConverter'], ['com.thoughtworks.xstream.converters.reflection.ReflectionProvider', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.reflection.ReflectionConverter', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.converters.extended.ToAttributedValueConverter', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.annotations.XStreamConverter'], ['com.thoughtworks.xstream.converters.SingleValueConverter', 'java.util.ArrayList', 'java.util.Collection', 'java.util.List'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'xstream.Content', 'org.apache.commons.lang.WordUtils', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['java.util.HashMap', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'java.util.Map', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'com.thoughtworks.xstream.io.xml.DomDriver', 'java.util.List'], ['com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.UnmarshallingContext', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.MarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamReader'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias'], ['com.thoughtworks.xstream.XStream', 'xstream.HibernateProxyConverter', 'xstream.HibernateMapper', 'xstream.HibernatePersistentCollectionConverter', 'xstream.HibernatePersistentMapConverter', 'xstream.HibernatePersistentSortedMapConverter', 'xstream.HibernatePersistentSortedSetConverter', 'com.thoughtworks.xstream.mapper.MapperWrapper'], ['xstream.ASObject', 'xstream.ErrorMessage', 'com.thoughtworks.xstream.XStream', 'xstream.CommandMessageExt', 'xstream.RemotingMessage', 'xstream.MessageBody', 'com.thoughtworks.xstream.io.xml.DomDriver', 'xstream.ActionMessage', 'xstream.AcknowledgeMessage', 'xstream.AsyncMessage', 'xstream.AcknowledgeMessageExt', 'com.thoughtworks.xstream.mapper.Mapper', 'xstream.MessageHeader', 'xstream.CommandMessage'], ['com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'java.util.List', 'com.thoughtworks.xstream.annotations.XStreamImplicit', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.mapper.MapperWrapper', 'java.util.Iterator'], ['com.thoughtworks.xstream.converters.SingleValueConverter', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.mapper.ReflectionProvider', 'com.thoughtworks.xstream.converters.ConverterLookup', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.mapper.MapperWrapper'], ['com.thoughtworks.xstream.io.xml.StaxDriver', 'com.thoughtworks.xstream.XStream', 'java.util.HashMap', 'java.util.Map'], ['com.thoughtworks.xstream.io.MarshallingContext', 'com.thoughtworks.xstream.io.UnmarshallingContext', 'java.util.ArrayList', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'java.util.ArrayList', 'java.util.List'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.io.xml.XppDriver', 'java.io.Writer', 'com.thoughtworks.xstream.io.xml.PrettyPrintWriter'], ['com.thoughtworks.xstream.io.MarshallingContext', 'com.thoughtworks.xstream.io.UnmarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader'], ['com.thoughtworks.xstream.io.xml.StaxDriver', 'com.thoughtworks.xstream.XStream', 'java.util.List'], ['com.thoughtworks.xstream.annotations.XStreamImplicit', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'java.util.List'], ['com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'com.thoughtworks.xstream.io.xml.DomDriver', 'java.util.List'], ['java.io.FileOutputStream', 'com.thoughtworks.xstream.XStream', 'java.io.File']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

#### Self-consistency from 30 Sample

In [ ]:
# Android Zero-shot Self-consistency from 30 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-30-sample"
code_name = "android"
codes = android_codes
predicted_imports = [['android.app.Activity', 'android.widget.TextView', 'android.os.Bundle'], ['android.app.Activity', 'android.os.Bundle'], ['android.graphics.drawable.Drawable', 'com.google.android.maps.OverlayItem', 'com.google.android.maps.GeoPoint', 'com.google.android.maps.ItemizedOverlay'], ['android.app.Activity', 'android.widget.TextView', 'android.os.Bundle'], ['android.widget.TabHost', 'android.graphics.drawable.Drawable', 'android.widget.TabSpec'], ['android.widget.LinearLayout', 'android.view.ViewGroup', 'android.view.Gravity'], ['android.app.Activity', 'android.widget.ArrayAdapter', 'android.view.View', 'android.os.Bundle'], ['java.net.InetAddress', 'android.util.Log', 'java.net.Inet4Address', 'java.io.IOException', 'java.net.Socket', 'java.net.UnknownHostException'], ['java.util.Timer', 'android.widget.TextView', 'java.util.TimerTask', 'android.os.Bundle'], ['android.util.Log', 'android.os.SystemClock', 'android.widget.Chronometer'], ['android.graphics.Canvas', 'com.google.android.maps.MyLocationOverlay', 'android.location.Location', 'com.google.android.maps.GeoPoint', 'android.content.Context', 'com.google.android.maps.MapView'], ['android.widget.LinearLayout', 'android.graphics.Color', 'android.view.ViewGroup', 'android.os.Bundle', 'android.app.Activity', 'android.widget.TextView'], ['android.net.Uri', 'android.graphics.BitmapFactory', 'java.io.ByteArrayOutputStream', 'android.graphics.Bitmap', 'java.io.ByteArrayInputStream', 'java.io.IOException', 'android.content.ContentResolver', 'java.io.FileNotFoundException'], ['java.io.InputStream', 'java.net.MalformedURLException', 'java.io.FileOutputStream', 'java.net.HttpURLConnection', 'java.io.File', 'java.io.IOException', 'java.net.ProtocolException', 'java.net.URL', 'java.io.FileNotFoundException'], ['android.net.Uri', 'android.content.Intent'], ['android.graphics.Canvas', 'android.util.AttributeSet', 'android.util.Log', 'android.view.MotionEvent', 'android.widget.SeekBar', 'android.view.View', 'android.content.Context'], ['android.util.Log', 'org.apache.http.client.HttpClient', 'org.apache.http.impl.client.DefaultHttpClient', 'java.io.IOException', 'org.apache.http.HttpResponse', 'java.net.URISyntaxException', 'org.apache.http.client.methods.HttpGet', 'org.apache.http.HttpEntity', 'java.io.InputStreamReader', 'java.net.URI'], ['android.view.View.OnClickListener', 'android.content.Intent', 'android.content.DialogInterface', 'android.os.Bundle', 'android.app.Activity'], ['android.widget.GridView', 'android.widget.ImageView', 'android.view.View', 'android.view.ViewGroup', 'android.content.Context'], ['android.widget.TextView', 'android.view.View', 'android.content.Context', 'android.view.animation.ScaleAnimation'], ['android.widget.LinearLayout', 'android.content.Context', 'android.widget.ImageButton'], ['android.app.ListActivity', 'android.database.Cursor', 'android.provider.Contacts.People', 'android.content.Intent', 'android.view.View', 'android.widget.SimpleCursorAdapter', 'android.os.Bundle', 'android.widget.ListView'], ['android.widget.ArrayAdapter', 'java.net.MalformedURLException', 'java.util.Random', 'android.graphics.BitmapFactory', 'java.net.HttpURLConnection', 'android.graphics.Bitmap', 'android.widget.ImageView', 'java.io.IOException', 'java.util.ArrayList', 'android.os.Bundle', 'android.app.Activity', 'java.net.URL', 'android.widget.ListView'], ['android.location.LocationManager', 'android.util.Log', 'java.util.List', 'java.util.Iterator', 'android.os.Bundle', 'android.app.Activity', 'android.content.Context'], ['android.app.ListActivity', 'android.database.sqlite.SQLiteDatabase', 'android.os.Bundle', 'java.util.ArrayList', 'android.content.Context'], ['android.widget.ScrollView', 'android.view.View.OnTouchListener', 'android.view.MotionEvent', 'android.view.View', 'android.view.KeyEvent', 'android.content.Context'], ['android.widget.GridView', 'android.widget.TextView', 'android.view.View', 'android.widget.BaseAdapter', 'android.view.ViewGroup', 'android.content.Context'], ['android.content.Context', 'android.app.Application'], ['android.content.Intent', 'android.os.PowerManager', 'android.content.Context', 'android.os.IBinder', 'android.app.Service'], ['org.apache.http.util.EntityUtils', 'org.apache.http.HttpHost', 'org.apache.http.client.HttpClient', 'org.apache.http.impl.client.DefaultHttpClient', 'org.apache.http.HttpResponse', 'java.io.IOException', 'org.apache.http.HttpEntity', 'org.apache.http.client.methods.HttpGet'], ['android.os.PowerManager', 'android.content.Context'], ['java.io.File', 'java.io.IOException', 'android.media.MediaRecorder', 'android.os.Environment'], ['android.hardware.Camera', 'android.graphics.PixelFormat', 'android.hardware.Camera.Parameters', 'java.io.IOException', 'android.content.Context', 'android.view.SurfaceHolder'], ['android.app.TabActivity', 'android.view.View.OnClickListener', 'android.widget.TabHost.TabContentFactory', 'android.util.Log', 'android.widget.TabHost.OnTabChangeListener', 'android.widget.TabHost', 'android.view.View', 'android.os.Bundle', 'android.widget.ListView'], ['java.io.File', 'java.io.IOException', 'android.media.MediaRecorder'], ['android.content.ContentValues', 'android.database.sqlite.SQLiteDatabase'], ['android.app.Activity', 'android.os.Message', 'android.app.ProgressDialog', 'android.os.Handler'], ['java.io.FileInputStream', 'java.io.InputStream', 'java.net.MalformedURLException', 'java.net.HttpURLConnection', 'java.io.File', 'java.io.DataOutputStream', 'java.io.IOException', 'java.net.URL', 'java.io.DataInputStream'], ['com.google.android.maps.Overlay', 'com.google.android.maps.MyLocationOverlay', 'com.google.android.maps.MapActivity', 'android.os.Bundle', 'android.app.Activity', 'com.google.android.maps.MapController', 'com.google.android.maps.MapView'], ['android.view.View.OnClickListener', 'android.util.Log', 'android.widget.ImageView', 'android.widget.SlidingDrawer', 'android.view.View', 'android.os.Bundle', 'android.app.Activity'], ['android.graphics.Color', 'android.widget.TextView', 'android.view.View', 'android.content.Context'], ['java.io.FileOutputStream', 'android.widget.Button', 'android.media.MediaPlayer.OnBufferingUpdateListener', 'android.util.Log', 'android.view.SurfaceView', 'android.graphics.PixelFormat', 'android.media.MediaPlayer', 'android.media.MediaPlayer.OnErrorListener', 'android.content.Intent', 'android.app.Activity', 'android.content.pm.ActivityInfo', 'android.net.Uri', 'android.media.MediaPlayer.OnPreparedListener', 'java.io.IOException', 'android.view.View', 'android.os.Bundle', 'java.net.URL', 'android.webkit.URLUtil', 'java.io.InputStream', 'java.net.URLConnection', 'java.io.File', 'android.media.MediaPlayer.OnCompletionListener', 'android.view.SurfaceHolder'], ['org.apache.http.util.EntityUtils', 'org.apache.http.HttpHost', 'org.apache.http.client.HttpClient', 'org.apache.http.impl.client.DefaultHttpClient', 'org.apache.http.HttpResponse', 'java.io.IOException', 'org.apache.http.HttpEntity', 'org.apache.http.client.methods.HttpGet'], ['android.os.IBinder', 'android.app.Service', 'android.content.Intent'], ['android.webkit.MalformedMimeTypeException', 'android.util.Log', 'android.content.Intent', 'android.content.ComponentName', 'android.content.BroadcastReceiver', 'android.os.Bundle', 'android.app.Activity', 'android.content.Context', 'android.content.IntentFilter'], ['android.widget.EditText', 'android.widget.Button', 'android.content.Intent', 'android.view.View', 'android.os.Bundle', 'android.app.Activity'], ['org.apache.http.client.ClientProtocolException', 'org.apache.http.impl.client.DefaultHttpClient', 'java.io.IOException', 'android.os.Bundle', 'android.app.Activity', 'android.widget.TextView', 'org.apache.http.client.methods.HttpGet'], ['android.graphics.BitmapFactory', 'android.app.Dialog', 'android.hardware.Camera', 'android.graphics.Bitmap', 'android.widget.TextView', 'android.widget.ImageView', 'android.app.AlertDialog', 'android.content.Context', 'android.hardware.Camera.PictureCallback'], ['android.widget.LinearLayout', 'android.content.Context', 'android.util.AttributeSet', 'android.view.Gravity', 'androidExamples.R', 'android.view.LayoutInflater'], ['android.widget.EditText', 'android.widget.Button', 'android.util.Log', 'java.text.NumberFormat', 'android.view.View', 'android.os.Bundle', 'android.app.Activity', 'android.widget.TextView']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# JDK Zero-shot Self-consistency from 30 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-30-sample"
code_name = "jdk"
codes = jdk_codes
predicted_imports = [['java.awt.event.WindowAdapter', 'java.io.FileInputStream', 'java.awt.Graphics2D', 'java.io.ObjectInputStream', 'java.io.IOException', 'java.awt.Graphics', 'java.io.ObjectOutputStream', 'java.awt.Applet', 'java.awt.Container', 'java.awt.image.BufferedImage', 'java.awt.event.WindowEvent', 'java.awt.Frame', 'java.io.FileOutputStream', 'java.awt.Dimension', 'java.awt.Color'], ['java.awt', 'javax.swing.border', 'java.awt.image', 'java.util', 'javax.swing', 'java.awt.event', 'javax.swing.table'], ['java.math.RoundingMode', 'java.text.DecimalFormatSymbols', 'java.text.DecimalFormat', 'java.text.ParseException', 'java.util.Locale'], ['java.util.ArrayList', 'java.util.List', 'java.util.regex.Pattern', 'java.util.regex.Matcher'], ['java.util.Collections', 'java.util.LinkedHashMap', 'java.util.HashMap', 'java.util.List', 'java.util.Map', 'java.util.TreeMap', 'java.util.LinkedList', 'java.util.Comparator'], ['java.util', 'javax.swing', 'java.awt.event', 'java.awt'], ['java.util.Collections', 'java.util.Date', 'java.text.SimpleDateFormat', 'java.util.ArrayList', 'java.text.ParseException', 'java.util.Comparator'], [], ['java.sql.DriverManager', 'java.sql.Connection', 'java.sql.SQLException', 'java.util.Properties'], ['java.text.NumberFormat', 'java.util.Locale'], ['java.io.BufferedWriter', 'javax.swing.JFileChooser', 'java.io.FileWriter', 'java.io.File'], ['java.util.Date', 'java.text.SimpleDateFormat', 'java.util.Calendar', 'java.util.GregorianCalendar', 'java.text.ParseException'], ['javax.swing.UIManager', 'javax.swing.SwingUtilities', 'javax.swing.JTextArea', 'javax.swing.JFrame'], ['java.lang.reflect.Method'], ['java.util.HashMap'], ['java.util.ArrayList', 'java.util.prefs.Preferences', 'java.lang.reflect.InvocationTargetException', 'java.util.HashMap', 'java.util.List', 'java.lang.reflect.Method', 'java.util.Map'], ['java.util.Date', 'java.text.SimpleDateFormat', 'java.text.ParseException', 'java.time.temporal.TemporalAccessor', 'java.time.format.DateTimeFormatter'], ['java.lang.reflect.Array', 'java.lang.reflect.Field'], ['java.util.concurrent.ThreadLocalRandom', 'java.util.Iterator', 'java.util.ArrayList', 'java.util.List', 'java.util.Collection', 'java.util.NoSuchElementException', 'java.util.Random'], ['java.io.FileInputStream', 'java.io.IOException', 'java.io.DataOutputStream', 'java.util.ArrayList', 'java.io.BufferedInputStream', 'java.io.DataInputStream', 'java.io.FileOutputStream', 'java.io.BufferedOutputStream', 'java.io.File'], ['java.awt.event.ActionListener', 'java.awt.Image', 'java.awt.event.ActionEvent', 'java.awt.event.KeyEvent', 'javax.swing.JPanel', 'javax.swing.JLabel', 'javax.imageio.ImageIO', 'java.awt.BorderLayout', 'javax.swing.SwingUtilities', 'javax.swing.JOptionPane', 'java.awt.event.InputEvent', 'javax.swing.KeyStroke', 'javax.swing.JTree', 'javax.swing.JScrollPane', 'javax.swing.border.TitledBorder', 'java.awt.Component', 'java.awt.image.BufferedImage', 'javax.swing.JTextArea', 'javax.swing.JMenuItem', 'java.io.File', 'javax.swing.JMenuBar', 'javax.swing.ImageIcon', 'javax.swing.JFrame', 'javax.swing.JMenu'], ['java.beans.PropertyChangeEvent', 'javax.swing.JProgressBar', 'javax.swing.SwingWorker', 'java.awt.GridLayout', 'java.awt.EventQueue', 'java.text.DecimalFormat', 'java.util.List', 'java.awt.Container', 'javax.swing.JFrame', 'javax.swing.JLabel', 'java.beans.PropertyChangeListener'], []]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# Hibernate Zero-shot Self-consistency from 30 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-30-sample"
code_name = "hibernate"
codes = hibernate_codes
predicted_imports = [['org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.SessionFactory'], ['org.hibernate.SessionFactory', 'org.hibernate.Transaction', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration', 'hibernate.UserDAO', 'hibernate.User'], ['java.util.List', 'java.io.Serializable', 'javax.persistence.Column', 'javax.persistence.OneToMany', 'javax.persistence.Cache', 'javax.persistence.Id', 'javax.persistence.Entity', 'javax.persistence.ManyToOne', 'javax.persistence.JoinColumn', 'javax.persistence.CacheConcurrencyStrategy'], ['org.hibernate.dialect.function.StandardSQLFunction', 'org.hibernate.dialect.function.VarArgsSQLFunction', 'java.sql.Types', 'org.hibernate.type.Hibernate', 'org.hibernate.dialect.Dialect', 'org.hibernate.dialect.function.SQLFunctionTemplate'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'org.hibernate.HibernateException'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory'], ['java.net.MalformedURLException', 'java.util.Iterator', 'java.util.List', 'org.dom4j.DocumentException', 'org.hibernate.cfg.SchemaExport', 'org.hibernate.SessionFactory', 'java.io.File', 'org.dom4j.io.SAXReader', 'org.hibernate.HibernateException', 'org.hibernate.Transaction', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session', 'org.dom4j.Document', 'org.hibernate.engine.EntityMode'], ['org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.SessionFactory', 'org.hibernate.Session'], ['javax.persistence.MappedSuperclass', 'javax.persistence.Column', 'javax.validation.constraints.AssertTrue', 'javax.validation.constraints.NotNull', 'java.util.Date', 'javax.persistence.TemporalType', 'javax.persistence.Temporal', 'javax.persistence.Transient'], ['java.util.List', 'java.io.Serializable', 'org.hibernate.Session'], ['java.util.List', 'java.io.Serializable', 'org.hibernate.Session'], ['java.io.Serializable', 'javax.persistence.ForeignKey', 'javax.persistence.Column', 'javax.persistence.FetchType', 'javax.persistence.Table', 'javax.persistence.Id', 'javax.persistence.Entity', 'javax.persistence.ManyToOne', 'javax.persistence.JoinColumn'], ['org.hibernate.SessionFactory', 'org.hibernate.Transaction', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session', 'javax.faces.context.FacesContext'], ['java.util.List', 'javax.persistence.CascadeType', 'javax.persistence.JoinTable', 'org.hibernate.annotations.IndexColumn', 'java.io.Serializable', 'java.util.Collection', 'javax.persistence.ForeignKey', 'javax.persistence.Column', 'javax.persistence.ManyToMany', 'javax.persistence.OneToMany', 'javax.persistence.FetchType', 'javax.persistence.Table', 'javax.persistence.Id', 'javax.persistence.Entity', 'javax.persistence.ManyToOne', 'javax.persistence.JoinColumn'], ['javax.persistence.CascadeType', 'java.io.Serializable', 'java.util.Collection', 'javax.persistence.ForeignKey', 'javax.persistence.Column', 'javax.persistence.ManyToMany', 'javax.persistence.FetchType', 'javax.persistence.Table', 'javax.persistence.Id', 'javax.persistence.Entity', 'javax.persistence.ManyToOne', 'javax.persistence.JoinColumn'], ['java.sql.Blob', 'java.io.IOException', 'java.io.InputStream', 'org.hibernate.Hibernate', 'java.io.ByteArrayOutputStream', 'java.sql.SQLException', 'java.util.Date'], ['java.util.List', 'org.hibernate.Session', 'hibernate.Event'], [], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory'], ['javax.persistence.Column', 'javax.persistence.GeneratedValue', 'javax.persistence.Index', 'javax.persistence.Id', 'org.hibernate.annotations.Table', 'javax.persistence.Entity'], ['org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.SessionFactory'], ['org.hibernate.Transaction', 'org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'org.hibernate.Session'], ['org.hibernate.query.Query', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session', 'com.google.gwt.json.client.JSONObject'], ['org.hibernate.SessionFactory', 'org.hibernate.HibernateException', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration'], ['org.hibernate.cfg.Configuration', 'java.lang.reflect.Method', 'org.hibernate.SessionFactory', 'java.lang.reflect.InvocationHandler'], ['org.hibernate.annotations.FetchMode', 'javax.persistence.CascadeType', 'javax.persistence.GeneratedValue', 'org.hibernate.annotations.Fetch', 'javax.persistence.Id', 'javax.persistence.Entity', 'javax.persistence.OneToOne'], ['org.hibernate.annotations.FetchMode', 'org.hibernate.Query', 'java.util.List', 'javax.persistence.CascadeType', 'org.hibernate.SessionFactory', 'javax.persistence.GeneratedValue', 'org.hibernate.annotations.Fetch', 'org.hibernate.Session', 'org.hibernate.cfg.AnnotationConfiguration', 'javax.persistence.Id', 'javax.persistence.Entity', 'javax.persistence.OneToOne', 'org.hibernate.tool.hbm2ddl.SchemaExport'], ['hibernate.HibernateUtil', 'org.hibernate.Query', 'java.util.List', 'org.hibernate.Session'], ['java.util.List', 'java.util.ArrayList', 'java.io.Serializable', 'javax.persistence.IndexColumn', 'javax.persistence.Column', 'javax.persistence.GeneratedValue', 'javax.persistence.OneToMany', 'javax.persistence.Id', 'javax.persistence.GenerationType', 'javax.persistence.Entity', 'javax.persistence.ManyToOne', 'javax.persistence.JoinColumn'], ['org.hibernate.criterion.Projections', 'java.util.List', 'org.hibernate.Criteria', 'org.hibernate.SessionFactory', 'org.hibernate.Session'], ['javax.persistence.Basic', 'java.io.Serializable', 'javax.persistence.NamedQueries', 'javax.persistence.NamedQuery', 'javax.persistence.Column', 'javax.persistence.Table', 'javax.persistence.Id', 'javax.persistence.Entity'], ['java.util.List', 'org.hibernate.query.Query', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session'], ['javax.persistence.GeneratedValue', 'javax.persistence.Index', 'javax.persistence.Table', 'javax.persistence.Id', 'javax.persistence.Entity'], ['org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'org.hibernate.Session'], ['org.hibernate.Transaction', 'org.hibernate.Session'], ['org.hibernate.Query', 'java.util.Iterator', 'java.util.List', 'org.hibernate.HibernateException', 'org.apache.log4j.BasicConfigurator', 'org.hibernate.Transaction', 'org.hibernate.Session'], ['org.hibernate.Transaction', 'org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'org.hibernate.Session'], ['org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.SessionFactory', 'org.hibernate.Session', 'org.hibernate.HibernateException'], ['org.hibernate.Transaction', 'org.hibernate.cfg.Configuration', 'org.hibernate.SessionFactory', 'org.hibernate.Session'], ['java.util.List', 'java.io.Serializable', 'org.hibernate.SessionFactory', 'java.util.Map', 'org.hibernate.Transaction', 'org.hibernate.Session', 'org.springframework.transaction.annotation.Transactional'], ['java.util.Iterator', 'java.util.List', 'org.hibernate.query.Query', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session'], ['java.util.List', 'org.hibernate.Criteria', 'org.hibernate.criterion.Restrictions', 'org.hibernate.Transaction', 'org.hibernate.Session'], ['java.util.List', 'org.hibernate.SessionFactory', 'javax.naming.InitialContext', 'org.hibernate.criterion.Example', 'org.apache.commons.logging.Log', 'org.apache.commons.logging.LogFactory'], ['java.util.Properties', 'org.hibernate.SessionFactory', 'org.hibernate.cfg.Configuration', 'org.hibernate.Session', 'org.apache.log4j.Logger'], ['javax.servlet.http.HttpSession', 'org.hibernate.Query', 'java.util.Iterator', 'org.apache.struts.action.ActionMapping', 'java.util.ArrayList', 'org.apache.struts.actions.DispatchAction', 'java.util.HashSet', 'org.hibernate.SessionFactory', 'org.hibernate.Transaction', 'javax.servlet.http.HttpServletRequest', 'javax.servlet.http.HttpServletResponse', 'org.hibernate.Session', 'java.util.Set', 'org.apache.struts.action.ActionForm', 'org.apache.struts.action.ActionForward'], ['org.hibernate.cfg.AnnotationConfiguration', 'org.hibernate.SessionFactory', 'org.hibernate.Session'], ['org.hibernate.SessionFactory', 'org.hibernate.HibernateException', 'org.hibernate.JDBCException', 'org.hibernate.CacheMode', 'org.hibernate.Session'], ['org.hibernate.Transaction', 'hibernate.HibernateUtil', 'org.hibernate.SessionFactory', 'org.hibernate.Session'], ['hibernate.HibernateUtil', 'org.hibernate.SessionFactory', 'org.hibernate.Transaction', 'com.google.gwt.user.server.rpc.RemoteServiceServlet', 'hibernate.StatelessProxyStore', 'org.hibernate.Session', 'hibernate.GreetingService', 'hibernate.PersistentRemoteService', 'hibernate.PersistentBeanManager'], ['javax.persistence.Version', 'org.hibernate.annotations.NaturalId', 'javax.persistence.Entity', 'java.io.Serializable'], ['java.util.List', 'org.hibernate.SessionFactory', 'hibernate.Name', 'org.hibernate.Session']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# JodaTime Zero-shot Self-consistency from 30 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-30-sample"
code_name = "jodatime"
codes = jodatime_codes
predicted_imports = [['org.joda.time.format.ISODateTimeFormat', 'java.util.TimeZone', 'org.joda.time.DateTimeZone', 'org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter'], ['org.joda.time.DateTimeZone', 'org.joda.time.DateMidnight', 'org.joda.time.DateTime'], ['org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.Interval', 'org.joda.time.format.PeriodFormatter'], ['org.joda.time.Period', 'org.joda.time.ReadableInstant'], ['org.joda.time.DateTimeZone', 'org.joda.time.Chronology', 'org.joda.time.chrono.GJChronology', 'org.joda.time.DateTime'], ['org.joda.time.Duration', 'org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.PeriodType'], ['java.text.DateFormat', 'org.joda.time.Days', 'org.joda.time.DateTime', 'java.text.SimpleDateFormat', 'java.text.ParseException', 'java.util.Calendar', 'java.util.Date'], ['java.util.concurrent.TimeUnit', 'org.joda.time.Period', 'org.joda.time.PeriodType'], ['org.joda.time.DateTimeZone', 'java.util.TimeZone'], ['java.io.ObjectOutputStream', 'java.io.IOException', 'org.joda.time.DateTimeZone', 'java.io.Serializable', 'org.joda.time.chrono.ISOChronology', 'java.io.ObjectInputStream'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'java.text.ParseException', 'java.text.SimpleDateFormat'], ['java.text.ParseException', 'java.text.SimpleDateFormat', 'java.util.Date', 'org.joda.time.DateTime'], ['org.joda.time.Period', 'org.junit.Assert.assertEquals', 'org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.format.PeriodFormatter', 'org.junit.Test'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime'], ['org.joda.time.MutableDateTime', 'org.joda.time.DateTime'], ['org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatter', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime'], ['org.joda.time.DateTimeZone', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormatterBuilder'], ['org.joda.time.Instant', 'org.joda.time.Interval', 'org.joda.time.ReadableInstant', 'java.util.Date'], ['org.joda.time.DateTimeZone', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.DateTime'], ['java.text.DateFormat', 'org.joda.time.LocalDateTime', 'org.joda.time.DateTime', 'org.joda.time.DateTimeZone', 'java.text.SimpleDateFormat', 'java.text.ParseException', 'java.util.Date'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter', 'java.text.ParseException', 'org.joda.time.format.DateTimeFormat', 'java.util.Date'], ['org.joda.time.LocalDate', 'org.joda.time.DateTimeConstants'], ['org.joda.time.Period', 'org.joda.time.format.PeriodFormat', 'org.joda.time.PeriodType', 'org.joda.time.DateTime'], ['org.joda.time.DateMidnight', 'org.joda.time.LocalDateTime', 'org.joda.time.Days'], ['org.joda.time.Duration', 'org.joda.time.Period'], ['java.util.ArrayList', 'org.joda.time.DateTime'], ['org.joda.time.Days', 'org.joda.time.DateTime', 'java.util.ArrayList', 'org.joda.time.LocalDate', 'org.joda.time.format.DateTimeFormatter', 'java.util.List', 'org.joda.time.format.DateTimeFormat'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.DateTime'], ['org.joda.time.DateTimeZone', 'org.joda.time.DateTime'], ['org.joda.time.Weeks', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.DateTime'], ['org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatter'], ['java.util.Locale', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatter', 'org.joda.time.format.PeriodFormat'], ['org.joda.time.PeriodFormatter', 'org.joda.time.DateTime', 'org.joda.time.Date', 'org.joda.time.ReadablePeriod', 'org.joda.time.PeriodFormatterBuilder', 'org.joda.time.Period'], ['org.joda.time.Period', 'org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.format.ISOPeriodFormat', 'org.joda.time.format.PeriodFormatter'], ['org.joda.time.DateTimeZone', 'java.util.concurrent.TimeUnit', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime'], ['org.joda.time.DateTime'], ['org.joda.time.Days', 'java.util.Date', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime'], ['org.joda.time.LocalDate'], ['org.joda.time.DateTime', 'org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeParser', 'org.joda.time.format.DateTimeFormatterBuilder', 'org.joda.time.format.DateTimeFormat'], ['org.joda.time.format.PeriodFormatterBuilder', 'org.joda.time.Period', 'org.joda.time.format.PeriodFormatter'], ['java.util.Set', 'java.util.TimeZone', 'org.joda.time.DateTimeZone', 'org.joda.time.tz.Provider', 'java.util.HashSet', 'java.util.Date'], ['org.joda.time.DateTimeZone', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.DateTime'], ['org.joda.time.MillisProvider', 'org.joda.time.DateTime'], ['org.joda.time.format.DateTimeFormatter', 'org.joda.time.format.DateTimeFormat', 'org.joda.time.LocalDate', 'org.joda.time.DateTime'], ['org.joda.time.DateTime']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# GWT Zero-shot Self-consistency from 30 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-30-sample"
code_name = "gwt"
codes = gwt_codes
predicted_imports = [['com.google.gwt.event.dom.client.MouseDownHandler', 'com.google.gwt.event.dom.client.MouseDownEvent', 'com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.user.client.ui.Composite'], ['com.google.gwt.sample.stockwatcher.client.GreetingServiceAsync', 'com.google.gwt.sample.stockwatcher.client.GreetingService', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.junit.client.GWTTestCase', 'com.google.gwt.core.client.GWT'], ['com.google.gwt.user.client.Window', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.core.client.EntryPoint'], ['com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.VerticalSplitPanel', 'com.google.gwt.user.client.ui.Composite'], ['com.google.gwt.core.client.JsArrayInteger', 'com.google.gwt.core.client.JsArray'], ['com.google.gwt.user.client.DOM', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.Element'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.Timer', 'com.google.gwt.user.client.ui.Table', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.Panel'], ['com.google.gwt.http.client.RequestException', 'com.google.gwt.user.client.Window', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response'], ['com.google.gwt.user.client.Command', 'com.google.gwt.user.client.ui.MenuBar', 'com.google.gwt.user.client.ui.VerticalPanel'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickListener', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.FormPanel', 'com.google.gwt.event.dom.client.ClickEvent'], ['com.google.gwt.user.client.ui.Label', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.RootPanel'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.AbsolutePanel'], ['com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.ClickListener', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.AbsolutePanel'], ['com.google.gwt.user.client.Window', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.core.client.GWT'], ['com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.URL', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response'], ['com.google.gwt.user.client.ui.HTML', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response'], ['com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.ImageBundle', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.AbstractImagePrototype', 'com.google.gwt.user.client.ui.Image', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.DockPanel', 'com.google.gwt.user.client.ui.Composite'], ['com.google.gwt.uibinder.client.UiBinder', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.core.client.GWT', 'com.google.gwt.user.client.ui.Composite'], ['com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.VerticalPanel'], ['com.google.gwt.user.client.Window', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.event.dom.client.ClickEvent'], ['com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.PopupPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.VerticalPanel'], ['com.google.gwt.uibinder.client.UiBinder', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.uibinder.client.UiField', 'com.google.gwt.core.client.GWT', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.Composite'], ['com.google.gwt.http.client.RequestException', 'java.util.UUID', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response'], ['com.google.gwt.user.client.DOM', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.event.dom.client.HasClickHandlers', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.Composite'], ['com.google.gwt.user.client.ui.DialogBox', 'com.google.gwt.event.dom.client.KeyUpEvent', 'com.google.gwt.event.dom.client.KeyUpHandler', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.event.dom.client.KeyCodes', 'com.google.gwt.event.dom.client.ClickEvent'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.event.dom.client.FormSubmitCompleteEvent', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickListener', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.event.dom.client.FormSubmitEvent', 'com.google.gwt.event.dom.client.FormHandler', 'com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.FormPanel'], ['com.google.gwt.dom.client.ButtonElement', 'com.google.gwt.dom.client.NodeList', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.dom.client.Document', 'com.google.gwt.core.client.GWT'], ['com.google.gwt.http.client.RequestException', 'com.google.gwt.user.client.Window', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.core.client.EntryPoint'], ['com.google.gwt.user.client.Window', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.jsonp.client.JsonpRequestBuilder'], ['com.google.gwt.user.client.ui.TextBox', 'com.google.gwt.user.client.ui.DialogBox', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.Grid'], ['com.google.gwt.http.client.RequestException', 'com.google.gwt.http.client.URL', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response'], ['com.google.gwt.http.client.RequestException', 'com.google.gwt.user.client.Window', 'com.google.gwt.http.client.URL', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response'], ['com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.VerticalPanel', 'java.util.List', 'java.util.Iterator', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.CheckBox', 'com.google.gwt.user.client.ui.DockPanel', 'com.google.gwt.user.client.ui.Grid', 'com.google.gwt.user.client.ui.RadioButton', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.ClickListener', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.core.client.EntryPoint', 'java.util.ArrayList'], ['com.google.gwt.user.client.DOM', 'com.google.gwt.event.dom.client.SubmitHandler', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.SubmitEvent', 'com.google.gwt.user.client.ui.FormPanel', 'com.google.gwt.event.dom.client.ClickEvent'], ['com.google.gwt.animation.client.Animation', 'com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.dom.client.Element', 'com.google.gwt.user.client.ui.FlowPanel', 'com.google.gwt.core.client.GWT'], ['com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.AbsolutePanel', 'com.google.gwt.user.client.ui.SimplePanel'], ['com.google.gwt.http.client.RequestException', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.Window', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.core.client.EntryPoint'], ['com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Hyperlink', 'com.google.gwt.event.dom.client.HasClickHandlers', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.user.client.ui.Composite'], ['java.io.DataInputStream', 'javax.servlet.ServletContext', 'java.io.FileInputStream', 'javax.servlet.http.HttpServletResponse', 'java.io.File', 'com.google.gwt.user.server.rpc.RemoteServiceServlet', 'javax.servlet.ServletOutputStream'], ['com.google.gwt.user.client.ui.Label', 'com.sencha.gxt.widget.core.client.container.LayoutContainer', 'com.google.gwt.user.client.ui.HorizontalPanel'], ['com.google.gwt.user.client.Element', 'com.google.gwt.user.client.ui.VerticalPanel', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.Composite'], ['com.google.gwt.http.client.RequestException', 'com.google.gwt.user.client.Window', 'com.google.gwt.http.client.RequestBuilder', 'com.google.gwt.http.client.RequestCallback', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.http.client.Request', 'com.google.gwt.http.client.Response', 'com.google.gwt.core.client.EntryPoint'], ['com.google.gwt.event.dom.client.MouseOutEvent', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.event.dom.client.MouseOutHandler', 'com.google.gwt.user.client.ui.HorizontalPanel', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.user.client.ui.Composite'], ['com.google.gwt.core.client.RunAsyncCallback', 'com.google.gwt.user.client.ui.DockLayoutPanel', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.user.client.rpc.AsyncCallback', 'com.google.gwt.core.client.GWT', 'com.google.gwt.user.client.ui.ResizeComposite'], ['com.google.gwt.user.client.Window', 'com.google.gwt.user.client.ui.Button', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.user.client.ui.Label', 'com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.event.dom.client.ClickEvent'], ['com.google.gwt.user.client.ui.RootPanel', 'com.google.gwt.user.client.ui.DialogBox', 'com.google.gwt.user.client.ui.HTML', 'com.google.gwt.user.client.ui.VerticalPanel'], ['com.google.gwt.user.client.ui.HTML', 'com.google.gwt.dom.client.Style.Unit', 'com.google.gwt.user.client.ui.LayoutPanel', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.core.client.EntryPoint', 'com.google.gwt.user.client.ui.RootLayoutPanel'], ['com.google.gwt.event.logical.shared.BeforeSelectionHandler', 'com.google.gwt.user.client.Window', 'com.google.gwt.event.logical.shared.SelectionHandler', 'com.google.gwt.event.logical.shared.BeforeSelectionEvent', 'com.google.gwt.event.logical.shared.SelectionEvent', 'com.google.gwt.user.client.ui.TabBar', 'com.google.gwt.core.client.EntryPoint'], ['com.google.gwt.user.client.DOM', 'com.google.gwt.dom.client.BodyElement', 'com.google.gwt.dom.client.Element', 'com.google.gwt.dom.client.Document', 'java.util.ArrayList'], ['com.google.gwt.user.client.DOM', 'com.google.gwt.user.client.ui.HasKeyDownHandlers', 'com.google.gwt.event.shared.HandlerRegistration', 'com.google.gwt.event.dom.client.KeyDownEvent', 'com.google.gwt.user.client.ui.HasBlurHandlers', 'com.google.gwt.event.dom.client.ClickHandler', 'com.google.gwt.event.dom.client.BlurHandler', 'com.google.gwt.user.client.ui.ComplexPanel', 'com.google.gwt.event.dom.client.KeyDownHandler', 'com.google.gwt.user.client.ui.HasText', 'com.google.gwt.user.client.ui.HasHTML', 'com.google.gwt.user.client.ui.HasClickHandlers', 'com.google.gwt.user.client.ui.Widget', 'com.google.gwt.event.dom.client.ClickEvent', 'com.google.gwt.event.dom.client.BlurEvent']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
# XStream Zero-shot Self-consistency from 30 Sample

save_directory = "/home/azmain/code_for_compilation_test/from-30-sample"
code_name = "xstream"
codes = xstream_codes
predicted_imports = [['com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver', 'java.io.FileReader', 'java.io.IOException', 'java.io.BufferedReader'], ['java.util.ArrayList', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.json.JettisonMappedXmlDriver', 'java.util.List'], ['com.thoughtworks.xstream.XStream', 'java.util.ArrayList'], ['java.io.FileNotFoundException', 'com.thoughtworks.xstream.XStream', 'java.io.PrintWriter', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['java.io.FileOutputStream', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.xml.DomDriver', 'java.io.OutputStream', 'java.io.File'], ['com.thoughtworks.xstream.annotations.XStreamAsAttribute'], ['org.ksoap2.transport.HttpTransportSE', 'com.thoughtworks.xstream.XStream', 'org.ksoap2.serialization.SoapObject', 'com.thoughtworks.xstream.io.xml.DomDriver', 'android.widget.TextView', 'org.ksoap2.SoapEnvelope', 'android.app.Activity', 'android.os.Bundle', 'org.ksoap2.serialization.SoapSerializationEnvelope'], ['java.io.FileNotFoundException', 'com.thoughtworks.xstream.XStream', 'java.io.FileInputStream', 'com.thoughtworks.xstream.io.xml.DomDriver'], ['com.thoughtworks.xstream.io.xml.PrettyPrintWriter', 'java.io.StringWriter', 'com.thoughtworks.xstream.XStream'], ['com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'java.util.List', 'com.thoughtworks.xstream.converters.collections.CollectionConverter', 'com.thoughtworks.xstream.mapper.ClassAliasingMapper'], ['java.util.List', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.mapper.ClassAliasingMapper'], ['com.thoughtworks.xstream.converters.SingleValueConverter', 'com.thoughtworks.xstream.converters.enums.AbstractSingleValueConverter'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.collections.CollectionConverter', 'java.util.HashSet', 'com.thoughtworks.xstream.converters.MarshallingContext'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.mapper.MapperWrapper'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.MarshallingContext', 'com.thoughtworks.xstream.io.UnmarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.io.HierarchicalStreamReader'], ['java.util.HashMap', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'java.util.Map', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'java.io.InputStreamReader', 'java.io.Reader', 'java.nio.charset.Charset', 'java.io.FileInputStream', 'java.io.InputStream', 'java.io.File'], ['java.util.Map', 'com.thoughtworks.xstream.converters.reflection.ReflectionProvider', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.reflection.ReflectionConverter', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamConverter', 'java.util.Arrays', 'java.util.List', 'com.thoughtworks.xstream.converters.extended.ToAttributedValueConverter', 'com.thoughtworks.xstream.annotations.XStreamImplicit', 'java.io.File', 'com.thoughtworks.xstream.annotations.XStreamAlias'], ['com.thoughtworks.xstream.io.xml.XppDriver', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.converters.extended.JavaBeanConverter'], ['com.thoughtworks.xstream.converters.reflection.ReflectionProvider', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.reflection.ReflectionConverter', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.converters.extended.ToAttributedValueConverter', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.annotations.XStreamConverter'], ['com.thoughtworks.xstream.converters.SingleValueConverter', 'java.util.ArrayList', 'java.util.Collection', 'java.util.List'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'org.apache.commons.lang3.text.WordUtils', 'xstream.Content', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['java.util.HashMap', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'java.util.Map', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'com.thoughtworks.xstream.io.xml.DomDriver', 'java.util.List'], ['com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.UnmarshallingContext', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.MarshallingContext', 'com.thoughtworks.xstream.io.HierarchicalStreamReader'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias'], ['com.thoughtworks.xstream.hibernate.converter.HibernateProxyConverter', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.hibernate.converter.HibernatePersistentCollectionConverter', 'com.thoughtworks.xstream.hibernate.converter.HibernatePersistentSortedSetConverter', 'com.thoughtworks.xstream.hibernate.converter.HibernatePersistentMapConverter', 'com.thoughtworks.xstream.hibernate.mapper.HibernateMapper', 'com.thoughtworks.xstream.hibernate.converter.HibernatePersistentSortedMapConverter', 'com.thoughtworks.xstream.mapper.MapperWrapper'], ['xstream.ASObject', 'xstream.ErrorMessage', 'com.thoughtworks.xstream.XStream', 'xstream.CommandMessageExt', 'xstream.RemotingMessage', 'xstream.MessageBody', 'com.thoughtworks.xstream.io.xml.DomDriver', 'xstream.ActionMessage', 'xstream.AcknowledgeMessage', 'xstream.AsyncMessage', 'xstream.AcknowledgeMessageExt', 'com.thoughtworks.xstream.mapper.Mapper', 'xstream.MessageHeader', 'xstream.CommandMessage'], ['com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'java.util.List', 'com.thoughtworks.xstream.annotations.XStreamImplicit', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'com.thoughtworks.xstream.mapper.MapperWrapper', 'java.util.Iterator'], ['com.thoughtworks.xstream.converters.SingleValueConverter', 'com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.reflection.ReflectionProvider', 'com.thoughtworks.xstream.converters.ConverterLookup', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.mapper.Mapper', 'com.thoughtworks.xstream.mapper.MapperWrapper'], ['com.thoughtworks.xstream.io.xml.StaxDriver', 'com.thoughtworks.xstream.XStream', 'java.util.HashMap', 'java.util.Map'], ['com.thoughtworks.xstream.io.MarshallingContext', 'com.thoughtworks.xstream.io.UnmarshallingContext', 'java.util.ArrayList', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'java.util.ArrayList', 'java.util.List'], ['com.thoughtworks.xstream.XStream', 'com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.io.xml.XppDriver', 'java.io.Writer', 'com.thoughtworks.xstream.io.xml.PrettyPrintWriter'], ['com.thoughtworks.xstream.io.HierarchicalStreamWriter', 'com.thoughtworks.xstream.converters.Converter', 'com.thoughtworks.xstream.io.HierarchicalStreamReader', 'com.thoughtworks.xstream.converters.MarshallingContext', 'com.thoughtworks.xstream.converters.UnmarshallingContext'], ['com.thoughtworks.xstream.io.xml.StaxDriver', 'com.thoughtworks.xstream.XStream', 'java.util.List'], ['com.thoughtworks.xstream.annotations.XStreamImplicit', 'com.thoughtworks.xstream.annotations.XStreamAlias', 'java.util.List'], ['com.thoughtworks.xstream.XStream', 'java.util.ArrayList', 'com.thoughtworks.xstream.io.xml.DomDriver', 'java.util.List'], ['java.io.FileOutputStream', 'com.thoughtworks.xstream.XStream', 'java.io.File']]
import_list = group_imports(predicted_imports)
append_imports(save_directory, code_name, codes, import_list)